# FBI
---
 * 조원 : 서수아, 서선우, 김정우, 고원태 
 * 주제 : 중소형 상장기업 대상 기업정보의 동태적 분석을 통한 부실기업 예측
---

### 1. 라이브러리 및 데이터 로드


In [553]:
### 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings(action='ignore')

# 한글 깨짐 방지
plt.rcParams['font.family'] = 'Malgun Gothic'

In [554]:
# 데이터 불러오기
df_연결 = pd.read_csv('../datasets/raw_data_연결.csv', encoding='cp949')
df_개별 = pd.read_csv('../datasets/raw_data_개별.csv', encoding='cp949')

### 2. 데이터 기초통계량 및 결측치 확인

In [555]:
# 연결 데이터 확인
df_연결.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15801 entries, 0 to 15800
Data columns (total 141 columns):
 #    Column                             Dtype  
---   ------                             -----  
 0    회사명                                object 
 1    거래소코드                              int64  
 2    회계년도                               object 
 3    [제조]총자본증가율(IFRS연결)                 float64
 4    [제조]유형자산증가율(IFRS연결)                float64
 5    [제조]투자부동산증가율(IFRS연결)               float64
 6    [제조]비유동자산증가율(IFRS연결)               float64
 7    [제조]유동자산증가율(IFRS연결)                float64
 8    [제조]재고자산증가율(IFRS연결)                float64
 9    [제조]자기자본증가율(IFRS연결)                float64
 10   [제조]매출액증가율(IFRS연결)                 float64
 11   [제조]정상영업이익증가율(IFRS연결)              float64
 12   [제조]순이익증가율(IFRS연결)                 float64
 13   [제조]총포괄이익증가율(IFRS연결)               float64
 14   [제조]매출액총이익률(IFRS연결)                float64
 15   [제조]매출액정상영업이익률(IFRS연결)             float64
 16   [제

In [556]:
# 개별 데이터 확인
df_개별.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15801 entries, 0 to 15800
Data columns (total 141 columns):
 #    Column                           Dtype  
---   ------                           -----  
 0    회사명                              object 
 1    거래소코드                            int64  
 2    회계년도                             object 
 3    [제조]총자본증가율(IFRS)                 float64
 4    [제조]유형자산증가율(IFRS)                float64
 5    [제조]투자부동산증가율(IFRS)               float64
 6    [제조]비유동자산증가율(IFRS)               float64
 7    [제조]유동자산증가율(IFRS)                float64
 8    [제조]재고자산증가율(IFRS)                float64
 9    [제조]자기자본증가율(IFRS)                float64
 10   [제조]매출액증가율(IFRS)                 float64
 11   [제조]정상영업이익증가율(IFRS)              float64
 12   [제조]순이익증가율(IFRS)                 float64
 13   [제조]총포괄이익증가율(IFRS)               float64
 14   [제조]매출액총이익률(IFRS)                float64
 15   [제조]매출액정상영업이익률(IFRS)             float64
 16   [제조]매출액순이익률(IFRS)                float

In [557]:
# 연결 데이터 결측치 확인
df_연결.isna().sum()

회사명                            0
거래소코드                          0
회계년도                           0
[제조]총자본증가율(IFRS연결)          2986
[제조]유형자산증가율(IFRS연결)         2986
                           ...  
[제조]감가상각비(IFRS연결)           2986
[제조]종업원1인당 부가가치(IFRS연결)     2986
[제조]총자본투자효율(IFRS연결).1      15801
[제조]기계투자효율(IFRS연결).1       15801
[제조]부가가치율(IFRS연결).1        15801
Length: 141, dtype: int64

In [558]:
# 개별 데이터 결측치 확인
df_개별.isna().sum()

회사명                          0
거래소코드                        0
회계년도                         0
[제조]총자본증가율(IFRS)          1123
[제조]유형자산증가율(IFRS)         1123
                         ...  
[제조]감가상각비(IFRS)           1124
[제조]종업원1인당 부가가치(IFRS)     1124
[제조]총자본투자효율(IFRS).1      15801
[제조]기계투자효율(IFRS).1       15801
[제조]부가가치율(IFRS).1        15801
Length: 141, dtype: int64

In [559]:
## 전부 결측치인 컬럼 삭제
df_연결.drop(['[제조]총자본투자효율(IFRS연결).1', '[제조]기계투자효율(IFRS연결).1', '[제조]부가가치율(IFRS연결).1', '[제조]종업원1인당 부가가치(IFRS연결)'], axis=1, inplace=True)
df_개별.drop(['[제조]총자본투자효율(IFRS).1', '[제조]기계투자효율(IFRS).1', '[제조]부가가치율(IFRS).1', '[제조]종업원1인당 부가가치(IFRS)'], axis=1, inplace=True)

In [560]:
df_개별.head()

,회사명,거래소코드,회계년도,[제조]총자본증가율(IFRS),[제조]유형자산증가율(IFRS),[제조]투자부동산증가율(IFRS),[제조]비유동자산증가율(IFRS),[제조]유동자산증가율(IFRS),[제조]재고자산증가율(IFRS),[제조]자기자본증가율(IFRS),...,[제조]노동소득분배율(IFRS),[제조]자본분배율(IFRS),[제조]이윤분배율(IFRS),[제조]부가가치(IFRS),[제조]법인세비용차감전(계속사업)손익(IFRS),[제조]인건비(IFRS),[제조]금융비용(IFRS),[제조]임차료(IFRS),[제조]세금과공과(IFRS),[제조]감가상각비(IFRS)
0,(주)CMG제약,58820,2011/12,2.89,-6.44,0.0,-10.56,17.05,-1.43,-0.62,...,3669.25,-1264.49,3110.72,143.09,339.77,0.0,1340.15,27.18,13.32,137.59
1,(주)CMG제약,58820,2012/12,106.87,-4.02,0.0,-19.28,208.38,7.73,72.78,...,-88.04,-6892.74,4677.39,399.66,335.12,0.0,1392.52,-0.66,-0.15,-3.53
2,(주)CMG제약,58820,2013/12,-1.80,16.63,0.0,78.54,-18.73,56.29,55.20,...,6952.08,350.27,4971.13,327.28,196.12,0.0,1107.28,51.12,12.42,287.63
3,(주)CMG제약,58820,2014/12,3.87,74.30,0.0,68.15,-25.87,6.37,0.04,...,7065.04,171.50,5599.85,-157.09,215.13,0.0,1464.84,43.08,12.15,132.13
4,(주)CMG제약,58820,2015/12,-2.64,-5.37,0.0,1.66,-7.15,11.04,-4.47,...,4316.80,-3648.67,6218.50,-29.83,195.67,0.0,1890.05,27.67,7.63,90.95


In [561]:
## 컬럼명 통일 (combine 함수 사용 목적)
df_연결.columns = ['회사명', '거래소코드', '회계년도', '총자본증가율', '유형자산증가율', '투자부동산증가율', '비유동자산증가율',
                      '유동자산증가율', '재고자산증가율', '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율', 
                      '매출액총이익률', '매출액정상영업이익률', '매출액순이익률', '총자본사업이익률', '총자본정상영업이익률', '총자본순이익률', '자기자본정상영업이익률',
                      '자기자본순이익률', '경영자본정상영업이익률', '경영자본순이익률', '자본금정상영업이익률', '자본금순이익률', '매출원가 대 매출액비율',
                      '영업비용 대 영업수익비율', '기타손익비률', '금융손익비율', '금융비용부담률', '외환이익 대 매출액비율', '광고선전비 대 매출액비율',
                      '세금과공과 대 세금과공과 차감전순이익률', '기업순이익률', '수지비율(관계기업투자손익 제외)', '인건비 대 총비용비율', 'R & D 투자효율',
                      '세금과공과 대 총비용비율', '금융비용 대 총비용비율', '감가상각비 대 총비용비율', '감가상각률', '누적감가상각률', '이자부담률', '지급이자율',
                      '차입금평균이자율', '유보율', '사내유보율', '사내유보 대 자기자본비율', '적립금비율(재정비율)', '평균배당률', '자기자본배당률',
                      '배당성향', '1주당매출액', '1주당순이익', '1주당 CASH FLOW', '1주당순자산', '1주당정상영업이익', '유동자산구성비율', '재고자산 대 유동자산비율',
                      '유동자산 대 비유동자산비율', '당좌자산구성비율', '비유동자산구성비율', '자기자본구성비율', '타인자본구성비율', '자기자본배율', '비유동비율',
                      '비유동장기적합률', '유동비율', '당좌비율', '현금비율', '매출채권비율', '재고자산 대 순운전자본비율', '매출채권 대 매입채무비율', '매출채권 대 상',
                      '제품비율', '매입채무 대 재고자산비율', '부채비율', '유동부채비율', '단기차입금 대 총차입금비율', '비유동부채비율', '비유동부채 대 순운전자본비율',
                      '순운전자본비율', '차입금의존도', '차입금비율', '이자보상배율(이자비용)', '이자보상배율(순금융비용)', '유보액대비율', '유보액 대 납입자본배율', 
                      '유동자산집중도', '비유동자산집중도', '투자집중도', 'CASH FLOW 대 부채비율', ' CASH FLOW 대 차입금비율', 'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율',
                      '정상영업이익대비이자보상배율', '총자본회전률', '경영자본회전률', '자기자본회전률', '자본금회전률', '타인자본회전률', '매입채무회전률', '매입채무회전기간', '유동자산회전률',
                      '당좌자산회전률', '재고자산회전률', '재고자산회전기간', '상품', '제품회전률', '원', '부재료회전률', '재공품회전률', '매출채권회전률', '매출채권회전기간',
                      '비유동자산회전률', '유형자산회전율', '순운전자본회전률', '운전자본회전률', '1회전기간', '노동장비율','기계장비율', '자본집약도', '총자본투자효율', 
                      '설비투자효율', '기계투자효율', '부가가치율', '노동소득분배율', '자본분배율', '이윤분배율', '부가가치', '법인세비용차감전(계속사업)손익', '인건비', 
                      '금융비용', '임차료', '세금과공과', '감가상각비']

df_개별.columns = ['회사명', '거래소코드', '회계년도', '총자본증가율', '유형자산증가율', '투자부동산증가율', '비유동자산증가율',
                      '유동자산증가율', '재고자산증가율', '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율', 
                      '매출액총이익률', '매출액정상영업이익률', '매출액순이익률', '총자본사업이익률', '총자본정상영업이익률', '총자본순이익률', '자기자본정상영업이익률',
                      '자기자본순이익률', '경영자본정상영업이익률', '경영자본순이익률', '자본금정상영업이익률', '자본금순이익률', '매출원가 대 매출액비율',
                      '영업비용 대 영업수익비율', '기타손익비률', '금융손익비율', '금융비용부담률', '외환이익 대 매출액비율', '광고선전비 대 매출액비율',
                      '세금과공과 대 세금과공과 차감전순이익률', '기업순이익률', '수지비율(관계기업투자손익 제외)', '인건비 대 총비용비율', 'R & D 투자효율',
                      '세금과공과 대 총비용비율', '금융비용 대 총비용비율', '감가상각비 대 총비용비율', '감가상각률', '누적감가상각률', '이자부담률', '지급이자율',
                      '차입금평균이자율', '유보율', '사내유보율', '사내유보 대 자기자본비율', '적립금비율(재정비율)', '평균배당률', '자기자본배당률',
                      '배당성향', '1주당매출액', '1주당순이익', '1주당 CASH FLOW', '1주당순자산', '1주당정상영업이익', '유동자산구성비율', '재고자산 대 유동자산비율',
                      '유동자산 대 비유동자산비율', '당좌자산구성비율', '비유동자산구성비율', '자기자본구성비율', '타인자본구성비율', '자기자본배율', '비유동비율',
                      '비유동장기적합률', '유동비율', '당좌비율', '현금비율', '매출채권비율', '재고자산 대 순운전자본비율', '매출채권 대 매입채무비율', '매출채권 대 상',
                      '제품비율', '매입채무 대 재고자산비율', '부채비율', '유동부채비율', '단기차입금 대 총차입금비율', '비유동부채비율', '비유동부채 대 순운전자본비율',
                      '순운전자본비율', '차입금의존도', '차입금비율', '이자보상배율(이자비용)', '이자보상배율(순금융비용)', '유보액대비율', '유보액 대 납입자본배율', 
                      '유동자산집중도', '비유동자산집중도', '투자집중도', 'CASH FLOW 대 부채비율', ' CASH FLOW 대 차입금비율', 'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율',
                      '정상영업이익대비이자보상배율', '총자본회전률', '경영자본회전률', '자기자본회전률', '자본금회전률', '타인자본회전률', '매입채무회전률', '매입채무회전기간', '유동자산회전률',
                      '당좌자산회전률', '재고자산회전률', '재고자산회전기간', '상품', '제품회전률', '원', '부재료회전률', '재공품회전률', '매출채권회전률', '매출채권회전기간',
                      '비유동자산회전률', '유형자산회전율', '순운전자본회전률', '운전자본회전률', '1회전기간', '노동장비율','기계장비율', '자본집약도', '총자본투자효율', 
                      '설비투자효율', '기계투자효율', '부가가치율', '노동소득분배율', '자본분배율', '이윤분배율', '부가가치', '법인세비용차감전(계속사업)손익', '인건비', 
                      '금융비용', '임차료', '세금과공과', '감가상각비']

### 3. 연결, 개별데이터 결합 후 전처리

In [562]:
# combine_first() 이용, 개별데이터 값으로 연결데이터 결측치 채움
df_통합 = df_연결.combine_first(df_개별)

In [563]:
# 통합 데이터 확인
df_통합.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15801 entries, 0 to 15800
Data columns (total 137 columns):
 #    Column                 Dtype  
---   ------                 -----  
 0    회사명                    object 
 1    거래소코드                  int64  
 2    회계년도                   object 
 3    총자본증가율                 float64
 4    유형자산증가율                float64
 5    투자부동산증가율               float64
 6    비유동자산증가율               float64
 7    유동자산증가율                float64
 8    재고자산증가율                float64
 9    자기자본증가율                float64
 10   매출액증가율                 float64
 11   정상영업이익증가율              float64
 12   순이익증가율                 float64
 13   총포괄이익증가율               float64
 14   매출액총이익률                float64
 15   매출액정상영업이익률             float64
 16   매출액순이익률                float64
 17   총자본사업이익률               float64
 18   총자본정상영업이익률             float64
 19   총자본순이익률                float64
 20   자기자본정상영업이익률            float64
 21   자기자본순이익률               float64
 2

In [564]:
# 통합 데이터 결측치 확인
df_통합.isna().sum()

회사명           0
거래소코드         0
회계년도          0
총자본증가율     1017
유형자산증가율    1017
           ... 
인건비        1017
금융비용       1017
임차료        1124
세금과공과      1017
감가상각비      1017
Length: 137, dtype: int64

In [565]:
# 통합 데이터 기초통계량
df_통합.describe()

,거래소코드,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,매출액증가율,정상영업이익증가율,...,노동소득분배율,자본분배율,이윤분배율,부가가치,법인세비용차감전(계속사업)손익,인건비,금융비용,임차료,세금과공과,감가상각비
count,15801.000000,14784.000000,14784.000000,14784.000000,14784.000000,14784.000000,14784.000000,14784.000000,14784.000000,14784.000000,...,1.478400e+04,1.478400e+04,1.478400e+04,1.478400e+04,1.478400e+04,14784.000000,1.478400e+04,14677.000000,14784.000000,1.478400e+04
mean,84565.184482,9.583247,37.915430,34.895344,13.729690,11.932803,69.777906,13.254098,12.437695,14.518985,...,2.648351e+05,7.062653e+04,1.128361e+05,8.734491e+03,8.187439e+03,1710.413308,6.422121e+04,123.611068,17.912307,4.629771e+06
std,110969.145029,37.515172,844.961413,711.397602,66.026302,64.747066,2780.119963,151.370720,130.504185,1976.762305,...,2.263913e+06,1.035837e+06,7.146010e+05,7.494026e+04,7.064773e+04,16719.597402,6.242956e+05,899.060926,24.586691,1.654149e+08
min,20.000000,-95.840000,-99.670000,-99.900000,-99.470000,-98.810000,-99.990000,-1068.120000,-99.850000,-190325.270000,...,-2.517832e+06,-4.063346e+06,-1.224634e+05,-2.676656e+06,-1.231070e+03,-944.640000,0.000000e+00,-48078.630000,-533.540000,-2.657909e+09
25%,16580.000000,-1.570000,-3.482500,-0.702500,-2.300000,-6.082500,-8.202500,-0.380000,-4.760000,-13.832500,...,1.085461e+04,-1.755843e+03,7.552500e+03,-1.800750e+02,0.000000e+00,0.000000,1.541515e+03,44.490000,10.417500,9.029000e+01
50%,53590.000000,3.400000,0.390000,0.000000,2.540000,2.505000,0.000000,4.090000,2.770000,0.000000,...,2.902200e+04,4.910720e+03,1.726521e+04,6.795150e+02,0.000000e+00,0.000000,4.094275e+03,88.390000,17.545000,2.601700e+02
75%,104200.000000,13.340000,12.452500,0.000000,14.782500,17.210000,19.900000,13.120000,14.870000,16.010000,...,8.652496e+04,2.024711e+04,4.690448e+04,3.232243e+03,4.038875e+02,0.000000,1.381969e+04,143.870000,25.612500,1.055790e+03
max,950220.000000,1720.860000,83638.120000,52203.580000,2527.750000,3110.470000,313488.540000,14501.350000,11172.240000,116066.340000,...,1.136701e+08,6.115996e+07,2.811425e+07,2.189869e+06,1.840305e+06,540350.000000,2.959764e+07,45285.310000,1101.900000,1.339152e+10


In [566]:
# 회계년도 확인
df_통합['회계년도'].unique()

array(['2011/12', '2012/12', '2013/12', '2014/12', '2015/12', '2016/12',
       '2017/12', '2018/12', '2019/12', '2011/11', '2011/09', '2012/09',
       '2013/09', '2014/09', '2015/09', '2016/09', '2017/09', '2018/09',
       '2019/09', '2011/03', '2012/03', '2013/03', '2014/03', '2015/03',
       '2016/03', '2017/03', '2017/06', '2018/06', '2019/06', '2011/06',
       '2012/06', '2013/06', '2014/06', '2015/06', '2016/06', '2018/03',
       '2019/03', '2017/02', '2018/02', '2019/02', '2011/08', '2012/08',
       '2013/08', '2014/08', '2015/08', '2016/08', '2017/08', '2018/08',
       '2019/08', '2011/10', '2012/11', '2013/11', '2014/11', '2015/11',
       '2016/11', '2017/11', '2018/11', '2019/11'], dtype=object)

In [567]:
# 결산월 12월만 남기기
df_통합수정 = df_통합.drop(df_통합[(df_통합['회계년도'] != '2011/12') & (df_통합['회계년도'] != '2012/12') & (df_통합['회계년도'] != '2013/12') & 
                      (df_통합['회계년도'] != '2014/12') & (df_통합['회계년도'] != '2015/12') & (df_통합['회계년도'] != '2016/12') & 
                      (df_통합['회계년도'] != '2017/12') & (df_통합['회계년도'] != '2018/12') & (df_통합['회계년도'] != '2019/12')].index)

In [568]:
# 결산월 확인
df_통합수정['회계년도'].value_counts()

2017/12    1771
2018/12    1766
2019/12    1766
2016/12    1753
2015/12    1729
2014/12    1689
2013/12    1664
2012/12    1646
2011/12    1641
Name: 회계년도, dtype: int64

In [569]:
# 회계년도 월 삭제 & 숫자형으로 변환
df_통합수정['회계년도'] = df_통합수정['회계년도'].apply(lambda x: x.replace('/12', ''))
df_통합수정['회계년도'] = df_통합수정['회계년도'].astype('int64')

In [570]:
# 중복 값 찾기
df_통합수정[df_통합수정.duplicated()]

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,...,노동소득분배율,자본분배율,이윤분배율,부가가치,법인세비용차감전(계속사업)손익,인건비,금융비용,임차료,세금과공과,감가상각비
1029,(주)더블유게임즈,192080,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1031,(주)더블유게임즈,192080,2015,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,55263.79,36465.15,19922.34,-1647.99,0.00,373.84,150.44,363.58,14.45,11733.49
1033,(주)더블유게임즈,192080,2016,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,72683.76,53399.65,23589.94,-5217.51,0.00,591.42,320.27,437.71,16.63,14175.84
1162,(주)동서,26960,2011,9.81,12.13,-1.12,40.07,-22.23,53.89,8.21,...,156005.52,130803.04,31951.01,-14714.36,0.00,0.00,7965.82,294.48,16.40,1023.52
1164,(주)동서,26960,2012,7.94,-2.78,72.51,-13.45,48.69,-19.33,9.86,...,182101.51,157021.23,32824.11,-14940.04,0.00,0.00,7196.22,272.86,17.73,1408.78
1166,(주)동서,26960,2013,8.36,2.49,5.44,6.17,10.79,-0.46,8.59,...,191093.43,160398.31,37721.73,-13472.98,0.00,0.00,6446.37,286.14,17.17,1266.91
1168,(주)동서,26960,2014,5.66,-4.03,-0.42,3.51,7.94,0.83,6.94,...,198261.31,163649.35,41700.25,-13159.13,0.00,0.00,6070.84,309.60,16.86,1455.45
1170,(주)동서,26960,2015,6.86,5.35,-0.37,6.16,7.57,13.12,6.07,...,193098.29,155996.78,41873.76,-10958.60,0.00,0.00,6186.34,280.27,15.37,1123.07
7363,(주)카카오,35720,2011,21.26,36.10,0.00,57.83,3.85,0.00,28.10,...,116824.46,127432.81,0.00,-10608.35,0.00,0.00,27014.69,84.86,20.91,228.46
8765,(주)포스코케미칼,3670,2011,29.80,45.15,0.00,45.54,25.32,-8.44,34.56,...,264867.77,120398.70,120100.93,-3770.38,18465.15,0.00,9673.38,197.62,53.78,670.13


In [571]:
# 중복 데이터 제거 (중복 데이터 중 첫번째만 살리기)
df_통합수정2 = df_통합수정.drop_duplicates(keep='first')

In [572]:
# 회사명 숫자 확인 (9개년 데이터라 최대 9개까지 존재해야 정상)
firm_name = pd.DataFrame(df_통합수정2['회사명'].value_counts())
firm_name[firm_name['회사명'] > 9]

,회사명
에스케이오션플랜트(주),18
(주)셀트리온,15
(주)카카오,14
(주)신성이엔지,14
삼성물산(주),13
에스케이(주),13
(주)포인트엔지니어링,12
(주)포스코케미칼,12
(주)자비스,12
(주)정다운,11


In [573]:
# 회사명 하나에 거래소 코드가 복수인 경우
check = df_통합수정2.groupby(['회사명'])['거래소코드'].nunique().reset_index()
check.rename(columns={'거래소코드':'거래소코드_개수'}, inplace=True)
check = check[check['거래소코드_개수']>1]
check

,회사명,거래소코드_개수
274,(주)본느,2
420,(주)신성이엔지,2
610,(주)엔케이맥스,2
806,(주)인산가,2
831,(주)자비스,2
836,(주)정다운,2
1085,(주)포인트엔지니어링,2
1256,나무기술(주),2
1419,삼성물산(주),2
1502,씨아이에스(주),2


- 에스케이오션플랜트(주), (주)신성이엔지, 삼성물산(주), 에스케이(주), (주)포인트엔지니어링, (주)자비스, (주)정다운, 나무기술(주), (주)인산가, (주)엔케이맥스, 씨아이에스(주)는 회사 명은 하나이나 거래소 코드가 두 개인 경우
- (주)셀트리온, (주)카카오, (주)포스코케미칼은 회계년도에 두 개의 자료가 있는 경우
- 거래소 코드가 두 개인 경우는 다시 확인 후 처리 결정
- 회계년도 한 해에 두 개의 자료가 있는 경우는 일일히 확인 후 두번째 컬럼 제거

In [574]:
# 거래소 코드 하나에 회사명이 복수인 경우
check_code = df_통합수정2.groupby(['거래소코드'])['회사명'].nunique().reset_index()
check_code[check_code['회사명']>1]

,거래소코드,회사명
678,36420,2


- 이 경우는 중간에 회사명이 바뀐 경우
- 거래소코드 36420 하나만 존재 확인

In [575]:
# 해당 회사명 확인
print(df_통합수정2[df_통합수정2['거래소코드']==36420]['회사명'].unique())

['(주)제이콘텐트리' '(주)콘텐트리중앙']


In [576]:
# 바뀐 회사명으로 통일
df_통합수정2.loc[df_통합수정2['회사명'] == '(주)제이콘텐트리', '회사명'] = '(주)콘텐트리중앙'

In [577]:
# 처리 후 재확인(거래소코드 하나에 회사명이 2개인 기업)
check_code = df_통합수정2.groupby(['거래소코드'])['회사명'].nunique().reset_index()
check_code[check_code['회사명']>1]

,거래소코드,회사명


In [578]:
# 산업 분류 코드 상 금융 기업 삭제, 거래소코드 2개 기업 삭제
companies_to_drop = ['(주)경동인베스트','(주)오리온홀딩스', '(주)효성', '미원홀딩스(주)', '에이치디씨(주)', '쿠쿠홀딩스(주)', '컬러레이홀딩스', '(주)에이플러스에셋어드바이저', '인카금융서비스(주)', '글로벌텍스프리(주)', 
                     '(주)본느', '(주)신성이엔지', '(주)엔케이맥스', '(주)인산가', '(주)자비스', '(주)정다운','(주)포인트엔지니어링', '나무기술(주)', '삼성물산(주)', '씨아이에스(주)', '에스케이(주)',
                     '에스케이오션플랜트(주)']

df_통합수정2.drop(df_통합수정2.loc[df_통합수정2['회사명'].isin(companies_to_drop)].index, inplace=True)

In [579]:
# 처리 후 확인(거래소코드 2개인 기업)
check = df_통합수정2.groupby(['회사명'])['거래소코드'].nunique().reset_index()
check.rename(columns={'거래소코드':'거래소코드_개수'}, inplace=True)
check = check[check['거래소코드_개수']>1]
check

,회사명,거래소코드_개수


* 중복데이터 기업명 : (주)셀트리온, (주)카카오, (주)포스코케미칼, (주)콘탠트리중앙

In [580]:
# 회계년도 한 해에 두 개의 자료가 있는 기업 확인 / 중복 컬럼 제거 후 재 확인
df_통합수정2[df_통합수정2['회사명'] == '(주)셀트리온']

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,...,노동소득분배율,자본분배율,이윤분배율,부가가치,법인세비용차감전(계속사업)손익,인건비,금융비용,임차료,세금과공과,감가상각비
3081,(주)셀트리온,68270,2011,23.43,17.68,55.44,26.51,12.43,76.37,15.20,...,234650.65,183385.24,39705.09,-6823.52,801.25,715.88,16965.40,360.07,16.67,2639.75
3082,(주)셀트리온,68270,2011,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,234650.65,183385.24,39705.09,-6823.52,801.25,715.88,16965.40,360.07,16.67,2639.75
3083,(주)셀트리온,68270,2012,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,291412.54,187566.76,56122.23,9244.68,1026.32,946.04,36679.19,332.36,16.82,2440.15
3084,(주)셀트리온,68270,2012,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,291412.54,187566.76,56122.23,9244.68,1026.32,946.04,36679.19,332.36,16.82,2440.15
3085,(주)셀트리온,68270,2013,14.17,-3.26,-3.15,23.47,-4.45,75.42,3.52,...,234764.20,115664.59,47863.33,21608.50,1044.44,1100.75,47710.47,238.94,11.87,2214.09
3086,(주)셀트리온,68270,2013,14.17,-3.26,-3.15,23.47,-4.45,75.42,3.52,...,234764.20,115664.59,47863.33,21608.50,1044.44,1100.75,47710.47,238.94,11.87,2214.09
3087,(주)셀트리온,68270,2014,17.38,25.37,0.00,7.17,43.79,63.41,14.74,...,332736.95,149111.20,75196.20,28158.55,1898.77,2093.42,76278.81,296.08,14.33,2358.47
3088,(주)셀트리온,68270,2014,17.38,25.37,0.00,7.17,43.79,63.41,14.74,...,332736.95,149111.20,75196.20,28158.55,1898.77,2093.42,76278.81,296.08,14.33,2358.47
3089,(주)셀트리온,68270,2015,18.34,-0.07,0.00,7.98,38.31,4.54,35.82,...,357257.30,163110.86,84885.07,22657.19,2211.66,2218.44,82174.08,353.92,13.00,480.46
3090,(주)셀트리온,68270,2015,18.34,-0.07,0.00,7.98,38.31,4.54,35.82,...,357257.30,163110.86,84885.07,22657.19,2211.66,2218.44,82174.08,353.92,13.00,480.46


In [581]:
# 인덱스 번호 이용, 중복 컬럼 제거
df_통합수정2.drop([3082, 3084, 3086, 3088, 3090, 3092], axis=0, inplace=True)

In [582]:
# 회계년도 한 해에 두 개의 자료가 있는 기업 확인 / 중복 컬럼 제거 후 재 확인
df_통합수정2[df_통합수정2['회사명'] == '(주)카카오']

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,...,노동소득분배율,자본분배율,이윤분배율,부가가치,법인세비용차감전(계속사업)손익,인건비,금융비용,임차료,세금과공과,감가상각비
7362,(주)카카오,35720,2011,21.26,36.10,0.0,57.83,3.85,0.00,28.10,...,116824.46,127432.81,0.00,-10608.35,0.00,0.00,27014.69,84.86,20.91,228.46
7364,(주)카카오,35720,2012,9.51,13.57,0.0,12.70,7.20,0.00,12.88,...,90004.05,100852.07,0.00,-10848.02,0.00,0.00,35788.79,62.19,14.71,166.03
7365,(주)카카오,35720,2012,9.51,13.57,0.0,12.70,7.20,0.00,12.88,...,90004.05,100852.07,0.00,-10848.02,0.00,0.00,35788.79,62.19,14.71,166.03
7366,(주)카카오,35720,2013,7.50,18.07,0.0,16.53,0.63,0.00,4.57,...,244728.52,84057.09,122729.98,-10106.97,8435.88,0.00,39612.55,134.43,37.21,411.15
7367,(주)카카오,35720,2013,7.50,18.07,0.0,16.53,0.63,0.00,4.57,...,244728.52,84057.09,122729.98,-10106.97,8435.88,0.00,39612.55,134.43,37.21,411.15
7368,(주)카카오,35720,2014,320.83,64.74,0.0,539.47,128.27,0.00,355.51,...,318121.02,168829.12,122872.21,-5953.64,7540.79,1776.66,23055.88,139.59,11.49,344.14
7369,(주)카카오,35720,2014,320.83,64.74,0.0,539.47,128.27,0.00,355.51,...,318121.02,168829.12,122872.21,-5953.64,7540.79,1776.66,23055.88,139.59,11.49,344.14
7370,(주)카카오,35720,2015,15.19,11.25,0.0,12.62,21.52,165.55,3.98,...,461885.24,109542.97,271440.06,-10199.77,12717.22,2992.78,75391.99,156.94,14.49,431.87
7371,(주)카카오,35720,2015,15.19,11.25,0.0,12.62,21.52,165.55,3.98,...,461885.24,109542.97,271440.06,-10199.77,12717.22,2992.78,75391.99,156.94,14.49,431.87
7372,(주)카카오,35720,2016,72.00,15.89,0.0,92.35,25.45,134.56,34.48,...,596947.82,100295.61,352605.98,4156.62,21928.41,4182.51,113778.70,141.61,10.89,543.09


In [583]:
# 인덱스 번호 이용, 중복 컬럼 제거
df_통합수정2.drop([7365, 7367, 7369, 7371, 7373], axis=0, inplace=True)

In [584]:
# 회계년도 한 해에 두 개의 자료가 있는 기업 확인 / 중복 컬럼 제거 후 재 확인
df_통합수정2[df_통합수정2['회사명'] == '(주)포스코케미칼']

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,...,노동소득분배율,자본분배율,이윤분배율,부가가치,법인세비용차감전(계속사업)손익,인건비,금융비용,임차료,세금과공과,감가상각비
8764,(주)포스코케미칼,3670,2011,29.80,45.15,0.00,45.54,25.32,-8.44,34.56,...,264867.77,120398.70,120100.93,-3770.38,18465.15,0.0,9673.38,197.62,53.78,670.13
8766,(주)포스코케미칼,3670,2012,9.72,16.33,-12.58,39.32,-0.07,3.04,20.42,...,246780.31,97292.54,123027.29,-4684.97,19703.93,0.0,11441.52,181.33,45.67,556.98
8768,(주)포스코케미칼,3670,2013,16.66,-1.88,0.00,74.15,-9.87,-8.86,12.67,...,230223.49,76435.00,125779.50,-2968.79,18251.26,0.0,12726.51,170.81,36.52,482.75
8770,(주)포스코케미칼,3670,2014,14.47,18.88,0.00,11.67,16.96,-6.48,13.83,...,258609.33,94790.11,131189.78,-982.94,19400.47,0.0,14211.91,185.00,35.84,343.22
8772,(주)포스코케미칼,3670,2015,-0.57,6.55,0.00,-0.52,-0.61,6.98,5.00,...,204878.11,46240.55,124992.89,-866.33,20001.71,0.0,14509.29,154.87,28.55,245.13
8774,(주)포스코케미칼,3670,2016,0.89,-5.70,0.00,-8.13,8.57,-21.29,6.10,...,226028.09,64763.21,129739.75,-967.38,18027.43,0.0,14465.09,183.35,31.23,281.19
8775,(주)포스코케미칼,3670,2016,0.89,-5.70,0.00,-8.13,8.57,-21.29,6.10,...,226028.09,64763.21,129739.75,-967.38,18027.43,0.0,14465.09,183.35,31.23,281.19
8776,(주)포스코케미칼,3670,2017,15.47,33.67,0.00,21.22,11.32,75.29,14.56,...,282415.66,125560.00,125256.08,-1632.29,19033.52,0.0,14198.35,192.19,33.79,285.43
8777,(주)포스코케미칼,3670,2017,15.47,33.67,0.00,21.22,11.32,75.29,14.56,...,282415.66,125560.00,125256.08,-1632.29,19033.52,0.0,14198.35,192.19,33.79,285.43
8778,(주)포스코케미칼,3670,2018,13.37,44.25,0.00,38.12,-6.05,10.45,16.32,...,354194.90,176599.53,142976.64,-1331.18,19604.13,0.0,16345.79,197.38,37.38,238.36


In [585]:
# 인덱스 번호 이용, 중복 컬럼 제거
df_통합수정2.drop([8775, 8777, 8779], axis=0, inplace=True)

In [586]:
# 회계년도 한 해에 두 개의 자료가 있는 기업 확인 / 중복 컬럼 제거 후 재 확인
df_통합수정2[df_통합수정2['회사명'] == '(주)콘텐트리중앙']

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,...,노동소득분배율,자본분배율,이윤분배율,부가가치,법인세비용차감전(계속사업)손익,인건비,금융비용,임차료,세금과공과,감가상각비
6923,(주)콘텐트리중앙,36420,2011,83.68,331.55,0.00,153.14,27.59,542.59,941.41,...,61038.06,16899.35,20175.12,6645.98,6688.69,0.0,10628.93,90.28,23.24,7336.28
6924,(주)콘텐트리중앙,36420,2012,95.88,110.44,-0.87,89.17,106.64,-3.97,14.05,...,131173.40,29389.99,34693.75,9854.75,36651.88,0.0,20583.03,116.32,25.49,377.98
6925,(주)콘텐트리중앙,36420,2013,-1.56,17.43,-19.07,2.39,-7.34,0.32,-7.93,...,120299.78,13780.33,38662.25,7256.34,37642.15,0.0,22958.72,28.30,23.75,325.62
6926,(주)콘텐트리중앙,36420,2014,-5.02,21.30,-0.91,2.51,-17.22,29.66,4.12,...,124754.51,22973.72,38735.31,3597.17,37499.07,0.0,22024.71,48.41,25.93,284.55
6927,(주)콘텐트리중앙,36420,2015,-5.22,14.32,-9.26,11.40,-38.59,92.66,-30.59,...,121611.32,30253.96,34534.80,3545.36,36324.51,0.0,16957.69,65.01,26.67,232.66
6928,(주)콘텐트리중앙,36420,2016,20.67,55.17,-0.55,15.71,38.73,64.19,34.41,...,122051.80,35065.95,31374.63,1219.54,40421.58,0.0,14002.91,10.96,22.18,147.82
6929,(주)콘텐트리중앙,36420,2017,20.31,-9.51,248.25,9.59,52.87,1.34,88.74,...,147996.33,24382.39,45382.73,4994.90,47366.19,0.0,25870.12,3.91,22.36,148.43
6930,(주)콘텐트리중앙,36420,2018,24.06,8.16,-0.47,5.63,64.17,39.11,112.72,...,192336.35,33559.63,60314.59,3493.38,52732.11,0.0,42236.65,11.02,23.42,198.84
7983,(주)콘텐트리중앙,36420,2011,83.68,331.55,0.00,153.14,27.59,542.59,941.41,...,61038.06,16899.35,20175.12,6645.98,6688.69,0.0,10628.93,90.28,23.24,7336.28
7984,(주)콘텐트리중앙,36420,2012,95.88,110.44,-0.87,89.17,106.64,-3.97,14.05,...,131173.40,29389.99,34693.75,9854.75,36651.88,0.0,20583.03,116.32,25.49,377.98


In [ ]:
# 인덱스 번호 이용, 중복 컬럼 제거
df_통합수정2.drop([6923, 6924, 6925, 6926, 6927, 6928, 6929, 6930], axis=0, inplace=True)

In [ ]:
# 회계년도에 두 개의 자료 존재하는 행 확인
firm_name_1 = pd.DataFrame(df_통합수정2['회사명'].value_counts())
firm_name_1[firm_name_1['회사명'] > 9]

,회사명


In [ ]:
# 1차 전처리 완료
df_통합수정2

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,...,노동소득분배율,자본분배율,이윤분배율,부가가치,법인세비용차감전(계속사업)손익,인건비,금융비용,임차료,세금과공과,감가상각비
0,(주)CMG제약,58820,2011,1.98,-5.90,0.00,-12.89,17.64,-1.43,-2.02,...,3472.17,-1508.28,3118.67,141.57,376.12,0.0,1344.09,27.18,12.72,1.285100e+02
1,(주)CMG제약,58820,2012,109.15,-4.12,0.00,-16.43,207.08,7.73,75.17,...,269.31,-6660.15,4712.51,411.52,401.09,0.0,1404.34,-0.66,0.47,1.067000e+01
2,(주)CMG제약,58820,2013,-2.09,16.09,0.00,77.05,-18.89,56.29,55.10,...,6994.90,311.19,4986.26,337.40,243.90,0.0,1116.14,51.12,12.51,2.894000e+02
3,(주)CMG제약,58820,2014,3.86,74.30,0.00,68.15,-25.93,6.37,0.01,...,7061.34,158.04,5599.85,-147.32,215.13,0.0,1464.84,43.08,12.16,1.320600e+02
4,(주)CMG제약,58820,2015,-2.64,-5.37,0.00,1.66,-7.17,11.04,-4.48,...,4315.37,-3650.09,6218.50,-29.83,195.67,0.0,1890.05,27.67,7.63,9.092000e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15796,흥아해운(주),3280,2015,21.93,49.36,7.85,32.18,-11.18,-12.92,3.39,...,103792.31,13509.26,52570.08,18751.33,0.00,0.0,18964.51,108.70,13.11,3.459744e+09
15797,흥아해운(주),3280,2016,21.08,38.84,2.88,30.68,-25.08,56.39,11.22,...,105291.39,-16576.37,70010.36,26701.98,0.00,0.0,25157.71,86.46,10.98,3.509713e+09
15798,흥아해운(주),3280,2017,-11.43,-16.50,-11.63,-14.38,13.29,11.18,-39.28,...,46072.27,-72565.48,60440.89,30688.04,0.00,0.0,27508.83,44.78,5.43,1.535742e+09
15799,흥아해운(주),3280,2018,-5.33,-12.07,70.56,-5.17,-6.33,-5.85,-37.42,...,34054.90,-91209.02,68757.28,32180.14,0.00,0.0,24326.58,2.41,4.24,1.135163e+09


In [ ]:
df_통합수정2.to_csv('../datasets/1차전처리.csv', index=0)

## 4. Y레이블 작업
---

### 4-1. 상장폐지 기업

* 자발적 상장폐지 (인수합병, 우회 상장, 시장이동) 제외


### 데이터 로드
---

In [ ]:
y_label = pd.read_csv('../datasets/first.csv')
code_연결 = pd.read_csv('../datasets/연결감사의견코드.csv', encoding='cp949')
code_개별 = pd.read_csv('../datasets/감사의견코드.csv', encoding='cp949')


In [ ]:
## 연결코드 결측치 확인
code_연결.isna().sum()

회사명                0
거래소코드              0
회계년도               0
감사의견코드(IFRS)    2983
dtype: int64

In [ ]:
## 개별코드 결측치 확인
code_개별.isna().sum()

회사명                0
거래소코드              0
회계년도               0
감사의견코드(IFRS)    1040
dtype: int64

In [ ]:
# 감사의견코드 결합
code_통합 = code_연결.combine_first(code_개별)

In [ ]:
## 통합코드 결측치 확인
code_통합.isna().sum()

회사명               0
거래소코드             0
회계년도              0
감사의견코드(IFRS)    928
dtype: int64

In [ ]:
# 결측치 확인
nan = code_통합[code_통합['감사의견코드(IFRS)'].isna()]
nan['회사명'].unique()

array(['(주)강스템바이오텍', '(주)강원에너지', '(주)그리티', '(주)그린플러스', '(주)금비',
       '(주)금호에이치티', '(주)나노브릭', '(주)나무가', '(주)네오크레마', '(주)넥스틴', '(주)노랑풍선',
       '(주)노바렉스', '(주)뉴파워프라즈마', '(주)대구백화점', '(주)대원', '(주)대창스틸',
       '(주)더네이쳐홀딩스', '(주)더코디', '(주)덕신하우징', '(주)덱스터스튜디오', '(주)동구바이오제약',
       '(주)두올', '(주)디알젬', '(주)디에이테크놀로지', '(주)디지캡', '(주)디케이앤디', '(주)디케이티',
       '(주)디티앤씨', '(주)디피앤케이', '(주)라파스', '(주)래몽래인', '(주)레이', '(주)리드',
       '(주)링크제니시스', '(주)마크로밀엠브레인', '(주)매직마이크로', '(주)메디아나', '(주)메카로',
       '(주)미래자원엠엘', '(주)미코바이오메드', '(주)미투온', '(주)바른손', '(주)방림', '(주)베셀',
       '(주)본느', '(주)브이원텍', '(주)비덴트', '(주)비씨월드제약', '(주)비엘', '(주)비지에프',
       '(주)비츠로셀', '(주)비츠로시스', '(주)비츠로테크', '(주)삼양옵틱스', '(주)삼에스코리아',
       '(주)서전기전', '(주)서진시스템', '(주)서플러스글로벌', '(주)석경에이티', '(주)선익시스템',
       '(주)세경하이테크', '(주)세원정공', '(주)세종메디칼', '(주)세진중공업', '(주)센트랄모텍',
       '(주)셀레믹스', '(주)셀리버리', '(주)셀트리온헬스케어', '(주)소프트센', '(주)솔루에타',
       '(주)솔트룩스', '(주)수젠텍', '(주)슈피겐코리아', '(주)스튜디오미르', '(주)신도기연', '(주)신스틸',
       '(주)신영와코루', '(주)신화콘텍', 

In [ ]:
# 감사의견코드 결측치 기업들 탐색
code_통합[code_통합['회사명'] == '(주)노랑풍선']

,회사명,거래소코드,회계년도,감사의견코드(IFRS)
616,(주)노랑풍선,104620,2011/12,NaN
617,(주)노랑풍선,104620,2012/12,NaN
618,(주)노랑풍선,104620,2013/12,NaN
619,(주)노랑풍선,104620,2014/12,NaN
620,(주)노랑풍선,104620,2015/12,UQ
621,(주)노랑풍선,104620,2016/12,UQ
622,(주)노랑풍선,104620,2017/12,UQ
623,(주)노랑풍선,104620,2018/12,UQ
624,(주)노랑풍선,104620,2019/12,UQ


In [ ]:
# 감사의견코드 DU, DS, AG (의견거절코드)만 뽑기
code_부실 = code_통합[code_통합['감사의견코드(IFRS)'].isin(['DU', 'DS', 'AG'])]
code_부실

,회사명,거래소코드,회계년도,감사의견코드(IFRS)
208,(주)골드앤에스,35290,2019/12,DS
544,(주)네오퍼플,28090,2012/12,DS
580,(주)넥솔론,110570,2016/12,DU
587,(주)넥스지,81970,2017/12,DS
824,(주)다이나믹디자인,145210,2018/12,DU
...,...,...,...,...
14140,중국고섬공고유한공사,950070,2012/12,DS
14148,중국원양자원유한공사,900050,2016/12,DU
14641,태산엘시디(주),36210,2013/12,DU
15387,한일건설(주),6440,2012/12,DU


In [ ]:
code_부실['회계년도'].unique()

array(['2019/12', '2012/12', '2016/12', '2017/12', '2018/12', '2013/12',
       '2011/12', '2014/12', '2015/12', '2013/03'], dtype=object)

In [ ]:
# 상장폐지 기업들 중 감사의견코드 부실의 이유로 거절인 경우 + KIND의 상장폐지 기업 리스트 중 부실 이유로 폐지된 기업의 리스트
# 로우 데이터 내 해당 기업들에게 부실 컬럼 안에 1의 레이블 부여
firm_list = ['(주)핸디소프트', '풍림산업(주)', '(주)클루넷', '(주)디에이치패션', '금강제강(주)', '(주)배명금속',
             '(주)엔케이바이오', '(주)넷웨이브', '(주)중앙건설', '(주)트라이써클', '이디디컴퍼니(주)', '(주)다함이텍',
             '(주)유에이블', '(주)디에스', '(주)지앤에스티', '글로스텍(주)', '(주)제너시스템즈', '(주)위다스',
             '(주)예당컴퍼니', '(주)테라리소스', '엠텍비젼(주)', '(주)나노트로닉스', '(주)디지텍시스템스', '(주)터보테크',
             '(주)신일건업', '삼환기업(주)', '(주)해피드림', '(주)씨앤케이인터내셔널', '(주)태창파로스', '(주)디아이디',
             'KCW(주)', '평산차업집단유한공사', '(주)승화프리텍', '(주)포스코플랜텍', '(주)엠제이비', '(주)씨엘인터내셔널',
             '(주)케이엔씨글로벌', '(주)넥솔론', '(주)비엔씨컴퍼니', '(주)엔에스브이', '아이이(주)', '(주)미래자원엠엘', '(주)위노바',
             '(주)썬코어', '(주)씨그널엔터테인먼트그룹', '(주)스틸플라워', '성지건설(주)', '(주)위너지스', '레이젠(주)', '(주)넥스지', 
             '(주)트레이스', '(주)우성아이비', '(주)지디', '(주)에프티이앤이', '(주)데코앤이', '(주)제이테크놀로지', '(주)리드', '(주)코썬바이오']

new_list = code_부실['회사명'].unique()

list_1 = pd.DataFrame(firm_list)
list_2 = pd.DataFrame(new_list)

In [ ]:
# 상장폐지 기업 리스트 / 부실 사유 감사의견 거절 기업 리스트 결합
a = list_1.append(list_2)
a

y_label['부실'] = ''

for i in y_label.index:
    if y_label.loc[i, '회사명'] in a.values:
        y_label.loc[i, '부실'] = 1
    else:
        y_label.loc[i, '부실'] = 0

In [ ]:
# 상장폐지 기업목록
a

,0
0,(주)핸디소프트
1,풍림산업(주)
2,(주)클루넷
3,(주)디에이치패션
4,금강제강(주)
...,...
86,중국고섬공고유한공사
87,중국원양자원유한공사
88,태산엘시디(주)
89,한일건설(주)


In [ ]:
# 중복값 확인
print(a.duplicated().sum())
a[a.duplicated()]

23


,0
2,(주)넥솔론
3,(주)넥스지
5,(주)데코앤이
7,(주)디에스
9,(주)디지텍시스템스
13,(주)리드
18,(주)스틸플라워
20,(주)신일건업
21,(주)썬코어
22,(주)씨그널엔터테인먼트그룹


In [ ]:
# 중복값 제거
a.drop_duplicates(keep='first', inplace=True)

In [ ]:
# 중복값 확인
a.duplicated().sum()

0

In [ ]:
a

,0
0,(주)핸디소프트
1,풍림산업(주)
2,(주)클루넷
3,(주)디에이치패션
4,금강제강(주)
...,...
86,중국고섬공고유한공사
87,중국원양자원유한공사
88,태산엘시디(주)
89,한일건설(주)


In [606]:
# 상폐기업 하나씩 체크
y_label[y_label['회사명'] == '흥아해운(주)']

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,...,자본분배율,이윤분배율,부가가치,법인세비용차감전(계속사업)손익,인건비,금융비용,임차료,세금과공과,감가상각비,부실
15304,흥아해운(주),3280,2011,-3.13,-0.34,1.03,1.75,-15.59,49.62,-17.19,...,-19679.63,30547.97,12495.87,0.0,0.0,13401.08,41.66,6.92,7.882146e+05,1
15305,흥아해운(주),3280,2012,0.49,-4.08,-7.34,3.33,-8.24,-20.78,10.07,...,16864.49,36929.46,13632.79,0.0,0.0,13841.28,98.51,15.23,3.174389e+06,1
15306,흥아해운(주),3280,2013,7.03,-7.30,-6.58,-5.03,48.81,12.95,24.61,...,22007.96,29850.13,14181.41,0.0,0.0,14631.71,103.40,14.13,5.737466e+06,1
15307,흥아해운(주),3280,2014,13.69,45.19,6.24,26.06,-13.68,-8.50,7.76,...,20934.22,43280.04,13705.97,0.0,0.0,12242.99,102.74,13.89,1.218393e+07,1
15308,흥아해운(주),3280,2015,21.93,49.36,7.85,32.18,-11.18,-12.92,3.39,...,13509.26,52570.08,18751.33,0.0,0.0,18964.51,108.70,13.11,3.459744e+09,1
15309,흥아해운(주),3280,2016,21.08,38.84,2.88,30.68,-25.08,56.39,11.22,...,-16576.37,70010.36,26701.98,0.0,0.0,25157.71,86.46,10.98,3.509713e+09,1
15310,흥아해운(주),3280,2017,-11.43,-16.50,-11.63,-14.38,13.29,11.18,-39.28,...,-72565.48,60440.89,30688.04,0.0,0.0,27508.83,44.78,5.43,1.535742e+09,1
15311,흥아해운(주),3280,2018,-5.33,-12.07,70.56,-5.17,-6.33,-5.85,-37.42,...,-91209.02,68757.28,32180.14,0.0,0.0,24326.58,2.41,4.24,1.135163e+09,1
15312,흥아해운(주),3280,2019,-46.38,-51.51,-97.68,-51.51,-13.42,-58.98,-57.73,...,-54638.04,32265.06,22368.67,0.0,0.0,23529.72,9.92,3.10,4.455416e+08,1


In [607]:
# 부실 사유 감사의견 거절 기업 리스트 확인
code_부실[code_부실['회사명'] == '흥아해운(주)']

,회사명,거래소코드,회계년도,감사의견코드(IFRS)
15800,흥아해운(주),3280,2019/12,DU


- 타겟 결측치 : (주)미래자원엠엘, (주)웹솔루스, (주)유디피

- *가 붙은 기업 : KIND확인 상폐기업(TS2000 코드로 분류되지 않음)

- 2011년 - (주)핸디소프트*
- 2012년 - (주)미성포리테크, 미리넷(주), (주)폴리플러스, 풍림산업(주)*, (주)클루넷*, (주)허메스홀딩스, (주)디에이치패션*, 금강제강(주)*, (주)배명금속*, (주)엔케이바이오*,
동양텔레콤(주), 연합과기공고유한공사, (주)인스프리트, (주)넷웨이브*, (주)중앙건설*
- 2013년 - (주)트라이써클*, (주)휴먼텍코리아, (주)삼우이엠씨, 이디디컴퍼니(주)*, 한일건설(주), (주)다함이텍 *, (주)유에이블 *, (주)디에스 *, (주)유일엔시스, (주)자유투어,
(주)에듀언스, (주)알앤엘바이오, (주)네오퍼플, (주)룩손에너지홀딩스, (주)지앤에스티*, 글로스텍(주)*, 우경(주), (주)제너시스템즈*, (주)지아이바이오, (주)위다스*, (주)예당컴퍼니*,
중국고섬공고유한공사
- 2014년 - (주)테라리소스*, (주)렉스엘이앤지, 엠텍비젼(주)*, 태산엘시디(주), (주)모린스, (주)나노트로닉스*, 쌍용건설(주), STX조선해양(주), (주)동양건설산업, (주)아라온테크,
에버테크노(주), (주)유니드코리아
- 2015년 - (주)디지텍시스템스*, (주)터보테크*, (주)로케트전기, 울트라건설(주), (주)신일건업*, 경남기업(주), 삼환기업(주)*, (주)영진코퍼레이션, (주)웹솔루스, (주)해피드림*,
잘만테크(주), (주)씨앤케이인터내셔널*, (주)태창파로스*, 에이스하이텍(주), (주)와이즈파워, (주)디아이디*, KCW(주)*, 평산차업집단유한공사*
- 2016년 - (주)승화프리텍*, (주)플렉스컴, (주)포스코플랜텍*, (주)아이디에스, 제이앤유글로벌, (주)엠제이비*, (주)씨엘인터내셔널*
- 2017년 - (주)케이엔씨글로벌*, (주)우전, (주)넥솔론*, (주)신양오라컴디스플레이, (주)비엔씨컴퍼니*, (주)에스에스컴텍, (주)보루네오가구, (주)엔에스브이*, 아이이(주)*, (주)미래자원엠엘 *
- 2018년 - (주)위노바 (2018.3월), (주)썬코어 (2018.3월), (주)씨그널엔터테인먼트그룹, (주)스틸플라워, 성지건설(주), (주)위너지스, 레이젠(주), (주)넥스지, (주)트레이스, (주)우성아이비, (주)지디
- 2019년 - (주)에프티이앤이, (주)데코앤이, (주)제이테크놀로지
- 2020년 - (주)리드, (주)코썬바이오 

In [608]:
# 상장폐지 2개년만 남기고 삭제
y_label = y_label.drop([9114, 9115, 9116, 9117, 9118, 9119, 9120, 9121, 9122, 1384, 6918, 9819, 5701, 
                    7988, 12831, 406, 1491, 7985, 7986, 3351, 3354, 11452, 11453, 9110, 9111,
                    3514, 3515, 7981, 7982, 1335, 1336, 9485, 9486, 14351, 3201, 3202, 3203, 8543,
                    8544, 8545, 5017, 5018, 5021, 3554, 3555, 3556, 7359, 7360, 7361, 7362, 556, 
                    557, 558, 559, 2242, 2243, 2244, 2245, 4700, 4701, 4702, 4705, 12098, 12099, 
                    12100, 12101, 1951, 1952, 1953, 1954, 5676, 5677, 5678, 5679, 5680, 3421, 3422,
                    3423, 3426, 3478, 3479, 3480, 3481, 3484, 3185, 3186, 3187, 3188, 3191, 11623,
                    11624, 11625, 11626, 11629, 5669, 5670, 5671, 5672, 5675, 10681, 10682, 10683,
                    10684, 10687, 562, 563, 564, 565, 568, 8099, 8100, 8101, 8102, 8105, 5439, 
                    5442, 6896, 6897, 6898, 6899, 6902, 4634, 4635, 4636, 4637, 4640, 4641, 1045,
                    1046, 1047, 1048, 1049, 1052, 6730, 6731, 6732, 6733, 6734, 6735, 1683, 1684, 
                    1685, 1686, 1687, 1688, 1689, 7675, 7676, 7677, 7678, 7679, 7680, 7681, 190, 
                    191, 192, 193, 194, 195, 198, 525, 798, 799, 800, 801, 802, 805, 806, 1188, 
                    1609, 1632, 1633, 1655, 1872, 1966, 2171, 2441, 3331, 3332, 3333, 3336, 3507,
                    3508, 3509, 3510, 3513, 3622, 3623, 3624, 3625, 3626, 3627, 3628, 3631, 3759,
                    3760, 3763, 3881, 3882, 3883, 3884, 3885, 3888, 3889, 3913, 3914, 3917, 3951,
                    4088, 4148, 4149, 4150, 4153, 4547, 4548, 4549, 4550, 4553, 4642, 5037, 5038,
                    5039, 5040, 5041, 5042, 5045, 5100, 5103, 5345, 5348, 5376, 5377, 5378, 5379, 
                    5380, 5381, 5384, 5647, 5650, 5796, 5797, 5798, 5860, 5861, 5862, 5863, 5866,
                    5935, 6217, 6218, 6219, 6220, 6221, 6224, 6225, 6289, 6290, 6291, 6292, 6293, 
                    6294, 6297, 6351, 6460, 6461, 6464, 6559, 6916, 7551, 7552, 7553, 7554, 7555,
                    7558, 7559, 8506, 8595, 8720, 8721, 8724, 8858, 8859, 8860, 8861, 8862, 8865,
                    8866, 9123, 9373, 9581, 9636, 9639, 10479, 10577, 10941, 11622, 11866, 11913,
                    11914, 11915, 11916, 11917, 11920, 11921, 12223, 12519, 12522, 12834, 12985, 
                    12986, 12987, 12988, 12989, 12990, 12991, 12993, 13080, 13083, 13512, 13515, 
                    13588, 13589, 13592, 13595, 13711, 13712, 13713, 13714, 13715, 13716, 13717, 
                    13718, 14198, 14917, 15304, 15305, 15306, 15307, 15308, 15309, 15312], axis=0)

In [609]:
y_label['부실'].value_counts()

0    14775
1      197
Name: 부실, dtype: int64

In [610]:
y_label.loc[y_label['부실'] == 1, '회사명'].nunique()

109

### 부실 사유 상장폐지 종목 레이블 추가 완료

In [611]:
y_label.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14972 entries, 0 to 15311
Data columns (total 138 columns):
 #    Column                 Non-Null Count  Dtype  
---   ------                 --------------  -----  
 0    회사명                    14972 non-null  object 
 1    거래소코드                  14972 non-null  int64  
 2    회계년도                   14972 non-null  int64  
 3    총자본증가율                 14016 non-null  float64
 4    유형자산증가율                14016 non-null  float64
 5    투자부동산증가율               14016 non-null  float64
 6    비유동자산증가율               14016 non-null  float64
 7    유동자산증가율                14016 non-null  float64
 8    재고자산증가율                14016 non-null  float64
 9    자기자본증가율                14016 non-null  float64
 10   매출액증가율                 14016 non-null  float64
 11   정상영업이익증가율              14016 non-null  float64
 12   순이익증가율                 14016 non-null  float64
 13   총포괄이익증가율               14016 non-null  float64
 14   매출액총이익률                14016 non-nul

### 4-2 관리종목

* KIND에서 2012년~2020년까지 관리종목 기업목록 추출 및 excel 제작
* 공시의무위반 제외

In [612]:
# 데이터 로드
df_관리종목목록 = pd.read_excel('../연도별 관리종목/통합.xlsx')

In [613]:
# 관리종목 목록 확인
df_관리종목목록

,날짜,종목,거래소코드
0,2012,(주)신일건업,14350
1,2012,(주)웅진,16880
2,2012,연합과기공고유한공사,900030
3,2012,중국고섬공고유한공사,950070
4,2012,(주)중앙건설,15110
...,...,...,...
311,2020,(주)코스온,69110
312,2020,(주)엔지스테크널러지,208860
313,2020,(주)코아시아옵틱스,196450
314,2020,(주)세동,53060


In [614]:
# 인덱스 정리
df_관리 = y_label.reset_index(drop=True)

In [615]:
# 관리종목목록 중복 제거
df_관리종목목록.drop_duplicates(subset='거래소코드', keep='first', inplace=True, ignore_index=True)

In [616]:
# 관리종목 제거 후 확인
df_관리종목목록

,날짜,종목,거래소코드
0,2012,(주)신일건업,14350
1,2012,(주)웅진,16880
2,2012,연합과기공고유한공사,900030
3,2012,중국고섬공고유한공사,950070
4,2012,(주)중앙건설,15110
...,...,...,...
283,2020,(주)코스온,69110
284,2020,(주)엔지스테크널러지,208860
285,2020,(주)코아시아옵틱스,196450
286,2020,(주)세동,53060


In [617]:
# 원본데이터와 같은 컬럼 생성
df_관리종목목록['지정년도'] = df_관리종목목록['날짜']
df_관리종목목록['회사명'] = df_관리종목목록['종목']
df_관리종목목록['관리종목'] = 1

In [618]:
df_관리종목목록.head(2)

,날짜,종목,거래소코드,지정년도,회사명,관리종목
0,2012,(주)신일건업,14350,2012,(주)신일건업,1
1,2012,(주)웅진,16880,2012,(주)웅진,1


In [619]:
df_관리.head(2)

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,...,자본분배율,이윤분배율,부가가치,법인세비용차감전(계속사업)손익,인건비,금융비용,임차료,세금과공과,감가상각비,부실
0,(주)CMG제약,58820,2011,1.98,-5.90,0.0,-12.89,17.64,-1.43,-2.02,...,-1508.28,3118.67,141.57,376.12,0.0,1344.09,27.18,12.72,128.51,0
1,(주)CMG제약,58820,2012,109.15,-4.12,0.0,-16.43,207.08,7.73,75.17,...,-6660.15,4712.51,411.52,401.09,0.0,1404.34,-0.66,0.47,10.67,0


In [620]:
# 원본데이터와 merge할 데이터 제작
df_관리종목목록2 = df_관리종목목록[['거래소코드', '지정년도', '관리종목']]

In [621]:
df_관리종목목록2.head(2)

,거래소코드,지정년도,관리종목
0,14350,2012,1
1,16880,2012,1


In [622]:
# 원본데이터와 merge
df_관리통합 = pd.merge(df_관리, df_관리종목목록2, how='left', on='거래소코드')

In [623]:
# 결합데이터 전처리

df_관리통합['지정년도'] = df_관리통합['지정년도'].fillna(9999).astype('int')
df_관리통합['회계년도'] = df_관리통합['회계년도'].astype('int')
df_관리통합['관리종목'] = df_관리통합['관리종목'].fillna(0).astype('int')

In [624]:
# 관리종목 컬럼에 1인 경우 부실(Y레이블)에 1을 넣어주는 반복문

for i in range(len(df_관리통합)):
    if df_관리통합['관리종목'][i] == 1:
        df_관리통합['부실'][i] = 1

In [625]:
# 데이터 불균형 확인(1차) * 2개년만 남기기 전

df_관리통합['부실'].value_counts()

0    13289
1     1683
Name: 부실, dtype: int64

In [626]:
# 관리종목 데이터프레임만 추출
df_extract = df_관리통합[df_관리통합['지정년도'] != 9999]

In [627]:
# 관리종목 지정 이후 삭제할 데이터 인덱스를 담는 반복문 (ex A기업 2015년 관리종목 지정, A기업의 2015년부터 존재하는 인덱스 번호를 리스트에 전부 담음)
# 삭제할 인덱스 번호 : j

ls = []

for i in df_extract.index:
    for j in df_extract.index:
        if df_extract['거래소코드'][i] == df_extract['거래소코드'][j]:
           if df_extract['지정년도'][i] <= df_extract['회계년도'][j]:
            ls.append(j)

In [628]:
# 리스트 데이터프레임화
drop_after = pd.DataFrame(ls)

# 리스트 중복값 삭제
drop_after.drop_duplicates(subset=0, keep='first', inplace=True, ignore_index=True)

In [629]:
# drop함수에 사용할 ls2 생성
drop_after_ls = []

for i in drop_after[0]:
    drop_after_ls.append(i)

In [630]:
# 원본데이터 복사
df_관리통합_copy = df_관리통합.copy()

In [631]:
# 원본데이터 관리종목 지정년도부터 그 이후 데이터 삭제
df_관리통합_copy.drop(drop_after_ls, inplace=True)

In [632]:
# 데이터 불균형 확인(2차) *지정년도 + 그 이후 데이터 삭제

df_관리통합_copy['부실'].value_counts()

0    13289
1     1071
Name: 부실, dtype: int64

In [633]:
# 제대로 삭제되었는지 확인

df_관리통합_copy[df_관리통합_copy['회사명'] == '흥아해운(주)']

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,...,부가가치,법인세비용차감전(계속사업)손익,인건비,금융비용,임차료,세금과공과,감가상각비,부실,지정년도,관리종목
14970,흥아해운(주),3280,2017,-11.43,-16.50,-11.63,-14.38,13.29,11.18,-39.28,...,30688.04,0.0,0.0,27508.83,44.78,5.43,1.535742e+09,1,2020,1
14971,흥아해운(주),3280,2018,-5.33,-12.07,70.56,-5.17,-6.33,-5.85,-37.42,...,32180.14,0.0,0.0,24326.58,2.41,4.24,1.135163e+09,1,2020,1


In [634]:
# 관리종목 지정 남겨놓을 인덱스를 담는 반복문 (ex A기업 2015년 관리종목 지정, A기업의 2013년, 2014년 인덱스번호를 리스트에 담음)
# 빈 리스트 생성 및 남겨놓을 인덱스 번호 append
# 남겨놓을 인덱스 번호 : j

ls_date = []

for i in df_extract.index:
    for j in df_extract.index:
        if df_extract['거래소코드'][i] == df_extract['거래소코드'][j]:
           if ((df_extract['지정년도'][i] - 2) == df_extract['회계년도'][j]) or ((df_extract['지정년도'][i] - 1) == df_extract['회계년도'][j]):
            ls_date.append(j)

In [635]:
# 리스트 데이터프레임화
ls_date2 = pd.DataFrame(ls_date)

# 리스트 중복값 삭제
ls_date2.drop_duplicates(subset=0, keep='first', inplace=True, ignore_index=True)

In [636]:
# 남겨놓을 인덱스를 ls3 리스트로 제작
ls3 = []

for i in ls_date2[0]:
    ls3.append(i)

In [637]:
# '유지' 컬럼 생성

df_관리통합_copy['유지'] = 0

for i in ls3:
    df_관리통합_copy['유지'][i] = 1

In [638]:
# 삭제할 리스트 생성
ls_delete = []

for i in df_관리통합_copy.index:
    if (df_관리통합_copy['관리종목'][i] == 1) and (df_관리통합_copy['유지'][i] == 0):
        ls_delete.append(i)

In [639]:
# 1 레이블링 이전 0 레이블링 데이터들 삭제
df_관리통합_copy.drop(ls_delete, inplace=True)

In [640]:
## 외국계기업 데이터 프레임 제작
MN = df_관리통합_copy[df_관리통합_copy['거래소코드'] >= 900000]

In [641]:
## 외국계기업 인덱스 기준 삭제
df_관리통합_copy.drop(MN.index, inplace=True)

In [200]:
# 데이터 분포 확인
df_관리통합_copy['부실'].value_counts()

0    13219
1      464
Name: 부실, dtype: int64

In [642]:
# 2차 로우데이터 파일 제작

raw_data = df_관리통합_copy.copy()

In [643]:
raw_data

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,투자부동산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,...,법인세비용차감전(계속사업)손익,인건비,금융비용,임차료,세금과공과,감가상각비,부실,지정년도,관리종목,유지
0,(주)CMG제약,58820,2011,1.98,-5.90,0.00,-12.89,17.64,-1.43,-2.02,...,376.12,0.0,1344.09,27.18,12.72,1.285100e+02,0,9999,0,0
1,(주)CMG제약,58820,2012,109.15,-4.12,0.00,-16.43,207.08,7.73,75.17,...,401.09,0.0,1404.34,-0.66,0.47,1.067000e+01,0,9999,0,0
2,(주)CMG제약,58820,2013,-2.09,16.09,0.00,77.05,-18.89,56.29,55.10,...,243.90,0.0,1116.14,51.12,12.51,2.894000e+02,0,9999,0,0
3,(주)CMG제약,58820,2014,3.86,74.30,0.00,68.15,-25.93,6.37,0.01,...,215.13,0.0,1464.84,43.08,12.16,1.320600e+02,0,9999,0,0
4,(주)CMG제약,58820,2015,-2.64,-5.37,0.00,1.66,-7.17,11.04,-4.48,...,195.67,0.0,1890.05,27.67,7.63,9.092000e+01,0,9999,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14966,휴젤(주),145020,2016,26.00,20.30,0.00,13.39,40.79,69.30,21.63,...,1156.40,0.0,4809.26,233.00,29.89,3.469500e+02,0,9999,0,0
14967,휴젤(주),145020,2017,178.94,0.28,0.00,-0.99,348.89,14.75,175.20,...,1588.83,0.0,6762.07,376.80,15.69,4.971200e+02,0,9999,0,0
14968,휴젤(주),145020,2018,11.86,2.78,0.00,64.95,0.80,40.62,4.60,...,1477.54,0.0,7491.31,288.73,14.06,5.187000e+02,0,9999,0,0
14969,휴젤(주),145020,2019,-2.57,15.05,0.00,8.70,-6.41,32.23,-3.88,...,506.26,0.0,9486.12,190.55,12.65,4.445900e+02,0,9999,0,0


In [644]:
# 인덱스 정리 및 필요없는 컬럼제거(관리종목 레이블링에 사용한 컬럼)

raw_data.reset_index(drop=True, inplace=True)

raw_data.drop(columns=['지정년도', '관리종목', '유지'], inplace=True)

In [645]:
# 2차 로우데이터 제작(관리종목 레이블링)

raw_data.to_csv('../datasets/2차전처리.csv', index=0)

### 5. 논문 공통 변수만 원본 데이터에 남기기(1차)
 * altman피쳐 및 기타 피쳐 부착 전

In [646]:
# 데이터 로드

raw_data2 = pd.read_csv('../datasets/2차전처리.csv')

In [647]:
# 공통변수 컬럼만 추출(27개)
raw_data3 = raw_data2[['회사명', '거래소코드', '회계년도', '자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)', '유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '부실']]

In [648]:
# 확인
raw_data3.head()

,회사명,거래소코드,회계년도,자기자본구성비율,설비투자효율,총자본투자효율,이자보상배율(이자비용),유동비율,당좌비율,부채비율,총자본정상영업이익률,매출액정상영업이익률,매출액순이익률,자기자본순이익률,매출채권회전률,재고자산회전률,총자본회전률,순운전자본비율,부실
0,(주)CMG제약,58820,2011,66.05,89.82,19.56,-9.650000e+00,200.97,165.03,42.33,2.37,3.61,-8.50,-8.28,1.77,56.81,0.68,9.08,0
1,(주)CMG제약,58820,2012,55.32,1749.87,1.66,-1.518000e+01,693.49,649.98,21.50,-9.96,-25.87,-40.97,-26.84,1.71,63.97,0.39,30.58,0
2,(주)CMG제약,58820,2013,87.63,71.28,34.29,1.920000e+00,694.71,610.74,11.23,1.40,3.88,1.45,0.74,2.03,67.75,0.37,2.53,0
3,(주)CMG제약,58820,2014,84.39,79.30,31.04,1.000000e+09,324.58,268.23,17.79,0.78,1.94,0.69,0.32,1.54,76.46,0.41,2.66,0
4,(주)CMG제약,58820,2015,82.80,144.10,15.96,1.000000e+09,296.85,235.21,18.91,-4.49,-9.51,-13.50,-7.62,1.48,69.98,0.47,0.01,0


In [649]:
# csv로 제작

raw_data3.to_csv('../datasets/3차전처리.csv', index=0)

### 6. 증가율 컬럼 제작

* 기존의 증가율 컬럼의 경우 잘못된 값이 들어가 있어 새롭게 제작함
* 매출액증가율, 총자본증가율, 순이익증가율, 영업이익증가율, 유동자산증가율, 유형자산증가율 총 6개

In [650]:
# 데이터 로드
change_rate_연결 = pd.read_csv('../datasets/증가율연결.csv', encoding='949')
change_rate_개별 = pd.read_csv('../datasets/증가율개별.csv', encoding='949')

In [651]:
# 연결, 개별데이터 컬럼이름 통일
change_rate_연결.columns = ['회사명', '거래소코드', '회계년도', '매출액', '자산','당기순이익', '계속영업이익', '유동자산','유형자산']
change_rate_개별.columns = ['회사명', '거래소코드', '회계년도', '매출액', '자산','당기순이익', '계속영업이익', '유동자산','유형자산']

In [652]:
# 연결데이터 결측치 확인
change_rate_연결.isna().sum()

회사명          0
거래소코드        0
회계년도         0
매출액       6609
자산        6609
당기순이익     6609
계속영업이익    6609
유동자산      6609
유형자산      6609
dtype: int64

In [653]:
# 개별데이터 결측치 확인
change_rate_개별.isna().sum()

회사명          0
거래소코드        0
회계년도         0
매출액       2092
자산        2092
당기순이익     2092
계속영업이익    2092
유동자산      2092
유형자산      2092
dtype: int64

In [654]:
# 연결, 개별데이터 결합

change_rate_통합 = change_rate_연결.combine_first(change_rate_개별)

In [655]:
change_rate_연결.describe()

,거래소코드,매출액,자산,당기순이익,계속영업이익,유동자산,유형자산
count,20505.000000,1.389600e+04,1.389600e+04,1.389600e+04,1.389600e+04,1.389600e+04,1.389600e+04
mean,91438.422336,1.443239e+09,1.800834e+09,6.136214e+07,5.935222e+07,6.664860e+08,6.562369e+08
std,110605.100043,8.086161e+09,1.104875e+10,8.269989e+08,8.241806e+08,4.177768e+09,4.928862e+09
min,20.000000,0.000000e+00,1.628840e+06,-4.069603e+09,-4.019893e+09,8.885730e+05,0.000000e+00
25%,17960.000000,6.138200e+07,8.568250e+07,-2.079096e+06,-1.989914e+06,4.199821e+07,1.638245e+07
50%,58610.000000,1.592831e+08,1.874436e+08,4.533800e+06,4.485673e+06,9.121204e+07,4.963863e+07
75%,119500.000000,5.036807e+08,5.562391e+08,1.816833e+07,1.784632e+07,2.478990e+08,1.795159e+08
max,950220.000000,2.437714e+11,3.525645e+11,4.434486e+10,4.434486e+10,1.813853e+11,1.647018e+11


In [656]:
# 결합 후 결측치 확인
change_rate_통합.isna().sum()

회사명          0
거래소코드        0
회계년도         0
매출액       1979
자산        1979
당기순이익     1979
계속영업이익    1979
유동자산      1979
유형자산      1979
dtype: int64

In [657]:
# 12월 결산월 이외의 데이터 드랍

change_rate_통합 = change_rate_통합.drop(change_rate_통합[(change_rate_통합['회계년도'] != '2010/12') &(change_rate_통합['회계년도'] != '2011/12') & (change_rate_통합['회계년도'] != '2012/12') & (change_rate_통합['회계년도'] != '2013/12') & 
                      (change_rate_통합['회계년도'] != '2014/12') & (change_rate_통합['회계년도'] != '2015/12') & (change_rate_통합['회계년도'] != '2016/12') & 
                      (change_rate_통합['회계년도'] != '2017/12') & (change_rate_통합['회계년도'] != '2018/12') & (change_rate_통합['회계년도'] != '2019/12')].index)

In [658]:
# 회계년도만 남기기

change_rate_통합['회계년도'] = change_rate_통합['회계년도'].apply(lambda x: x.replace('/12', ''))
change_rate_통합['회계년도'] = change_rate_통합['회계년도'].astype('int')

In [659]:
# 중복데이터 확인

change_rate_통합.drop_duplicates(keep='first', inplace=True)

In [660]:
# 증가율 계산

change_rate_통합['매출액증가율'] = change_rate_통합.groupby(['거래소코드'])['매출액'].pct_change()
change_rate_통합['총자본증가율'] = change_rate_통합.groupby(['거래소코드'])['자산'].pct_change()
change_rate_통합['순이익증가율'] = change_rate_통합.groupby(['거래소코드'])['당기순이익'].pct_change()
change_rate_통합['영업이익증가율'] = change_rate_통합.groupby(['거래소코드'])['계속영업이익'].pct_change()
change_rate_통합['유동자산증가율'] = change_rate_통합.groupby(['거래소코드'])['유동자산'].pct_change()
change_rate_통합['유형자산증가율'] = change_rate_통합.groupby(['거래소코드'])['유형자산'].pct_change()

In [661]:
# 증가율을 만든 후 필요없는 컬럼 삭제

change_rate_통합.drop(columns=['매출액', '자산', '당기순이익', '계속영업이익', '유동자산', '유형자산'],axis=1, inplace=True)

In [662]:
# 2010년도 데이터 삭제

change_rate_통합 = change_rate_통합[change_rate_통합['회계년도'] > 2010]

In [663]:
# 인덱스 정리 후 CSV파일로 제작

change_rate_통합_저장 = change_rate_통합.reset_index(drop=True)

change_rate_통합_저장.to_csv('../datasets/변화율.csv', index=0)

^^^^^^
# 다시 제작함 (증가율)

### 7. altman 변수 제작

* RETA = 이익잉여금 / 총자산
* EBTA = 법인세비용차감전순이익 / 총자산
* OM = 법인세비용차감전순이익 / 매출액

In [664]:
# 데이터 로드

df_altman연결 = pd.read_csv('../datasets/altman연결1.csv', encoding='cp949')
df_altman개별 = pd.read_csv('../datasets/altman개별1.csv', encoding='cp949')

In [665]:
# 연결, 개별데이터 컬럼명 통일

df_altman연결.columns = ['회사명', '거래소코드', '회계년도', '이익잉여금', '법인세비용차감전순이익', '자산', '부채', '매출액', '합계_기말인원', 'PER(최고)', 'PER(최저)', 'PBR(최고)', 'PBR(최저)', '종가']
df_altman개별.columns = ['회사명', '거래소코드', '회계년도', '이익잉여금', '법인세비용차감전순이익', '자산', '부채', '매출액', '합계_기말인원', 'PER(최고)', 'PER(최저)', 'PBR(최고)', 'PBR(최저)', '종가']

In [666]:
# 연결 데이터 결측치 확인
df_altman연결.isna().sum()

회사명               0
거래소코드             0
회계년도              0
이익잉여금          2092
법인세비용차감전순이익    6609
자산             6609
부채             6609
매출액            6609
합계_기말인원        3220
PER(최고)        2333
PER(최저)        2333
PBR(최고)        2333
PBR(최저)        2333
종가             3462
dtype: int64

In [251]:
# 개별 데이터 결측치 확인
df_altman개별.isna().sum()

회사명               0
거래소코드             0
회계년도              0
이익잉여금          2092
법인세비용차감전순이익    2092
자산             2092
부채             2092
매출액            2092
합계_기말인원        3220
PER(최고)        2333
PER(최저)        2333
PBR(최고)        2333
PBR(최저)        2333
종가             3462
dtype: int64

In [252]:
# combine_first() 이용, 개별데이터 값으로 연결데이터 결측치 채움
df_altman통합 = df_altman연결.combine_first(df_altman개별)

In [253]:
# 통합 데이터 결측치 확인
df_altman통합.isna().sum()

회사명               0
거래소코드             0
회계년도              0
이익잉여금          2082
법인세비용차감전순이익    1979
자산             1979
부채             1979
매출액            1979
합계_기말인원        3211
PER(최고)        2323
PER(최저)        2323
PBR(최고)        2323
PBR(최저)        2323
종가             3430
dtype: int64

In [255]:
# 회계년도 연도만 남기기
df_altman통합['회계년도'] = df_altman통합['회계년도'].str[:4]

In [256]:
# 중복값 제거

df_altman통합.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [258]:
# altman 변수 계산

df_altman통합['RETA'] = df_altman통합['이익잉여금'] / df_altman통합['자산']
df_altman통합['EBTA'] = df_altman통합['법인세비용차감전순이익'] / df_altman통합['자산']
df_altman통합['OM'] = df_altman통합['법인세비용차감전순이익'] / df_altman통합['매출액']

In [259]:
# 종업원 수 증가율 제작 데이터 복사

change_rate_종업원 = df_altman통합.copy()

In [260]:
# 종업원 수 증가율 제작 위해 결측치를 bfill로 채움

change_rate_종업원['합계_기말인원'] = change_rate_종업원.groupby(['회사명'])['합계_기말인원'].fillna(method='bfill')

In [262]:
# 남은 결측치 1로 채움

change_rate_종업원['합계_기말인원'] = change_rate_종업원['합계_기말인원'].fillna(1)

In [264]:
# 회사명으로 그룹화 후 종업원 증가율 계산

change_rate_종업원['종업원수증가율'] = change_rate_종업원.groupby(['회사명'])['합계_기말인원'].pct_change()

In [266]:
# 회계년도 10년 이후 데이터만 남기기

change_rate_종업원['회계년도'] = change_rate_종업원['회계년도'].astype(int)

altman_after = change_rate_종업원[change_rate_종업원['회계년도'] > 2010]

In [268]:
# 필요한 변수만 데이터프레임으로 받기

altman_data = altman_after[['회사명', '거래소코드', '회계년도', '종가', 'RETA', 'EBTA', 'OM', '종업원수증가율']]

In [269]:
altman_data.reset_index(drop=True, inplace=True)

altman_data.to_csv('../datasets/altman.csv', index=0)

# 8. 파생변수
---
* 거래량회전율 : 1년거래량 / 발행주식 총수 
* 수정주가 거래량 : 1년 거래량 / 수정종가
* 거래량증가율 : T-1기 1년거래량 - T기 1년거래량 / T-1기 1년거래량
* 로그시가총액 : np.log10(시가총액)
* 시가총액증가율 : T-1기 시가총액 - T기 시가총액 / T-1기 시가총액
* DOL : 영업이익변화율 / 매출액변화율 (1 ~ 무한대)
* DFL : 순이익변화율/ 영업이익변화율 (1 ~ 무한대)
* 종업원 수 증가율 : T-1기 종업원수 - T기 종업원수 / T-1기 종업원수
* 금융비용부담률 : 이자비용/매출액
* 고정비율 : 유형자산/총자산
* R&D비율 : 연구개발비/매출액
* 채무부담비율 : 재고자산회전율/매입채무회전율

### 8-1 시장정보 파생변수 제작

In [300]:
# 데이터 로드

df_월별거래량1 = pd.read_csv('../datasets/stock/월별거래량데이터.csv', encoding='cp949')
df_월별거래량2 = pd.read_csv('../datasets/stock/월별거래량데이터2.csv', encoding='cp949')

df_주식총수 = pd.read_csv('../datasets/stock_quantity.csv', index_col=0)

In [273]:
# 회계년도만 남기기

df_월별거래량1['회계년도'] = df_월별거래량1['회계년도'].str[:4]
df_월별거래량2['회계년도'] = df_월별거래량2['회계년도'].str[:4]

In [275]:
# 월별 거래량 통합

df_월별거래량통합 = pd.concat([df_월별거래량1, df_월별거래량2])

In [276]:
# 인덱스 정리 및 그룹화

df_월별거래량통합.reset_index(drop=True, inplace=True)

df_월별거래량그룹화 = df_월별거래량통합.groupby(['회사명', '회계년도', '거래소코드'])


In [279]:
df_1년거래량 = df_월별거래량그룹화.agg(sum).reset_index()

In [280]:
df_1년거래량

,회사명,회계년도,거래소코드,거래량,거래일수
0,3노드디지탈그룹유한공사,2010,900010,599345487,251
1,3노드디지탈그룹유한공사,2011,900010,695283628,248
2,3노드디지탈그룹유한공사,2012,900010,418147882,248
3,3노드디지탈그룹유한공사,2013,900010,35869989,106
4,AJ네트웍스보통주,2015,95570,7666331,90
...,...,...,...,...,...
19909,히스토스템,2010,36840,372204062,251
19910,히스토스템,2011,36840,38463293,173
19911,힘스,2017,238490,20074864,108
19912,힘스,2018,238490,40763372,244


In [282]:
# 필요한 컬럼만 추출
df_거래량 = df_1년거래량[['회사명','회계년도', '거래소코드', '거래량']]

In [283]:
# 액면가 데이터 받아오기(40개 csv파일)

# 데이터를 받아올 경로 지정
path = '../datasets/price/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')] # 경로안에서 .csv로 끝나는 파일들을 로드

# 데이터프레임 리스트 생성
price = []
for i in file_list_py:
    data = pd.read_csv(path + i, encoding='cp949')
    price.append(data)

# 하나의 리스트에 받아진 데이터들을 개별 데이터프레임으로 변환
for i in range(0, 40, 1):
    df_name = 'df' + str(i+1) 
    globals()[df_name] = pd.DataFrame(price[i])

In [285]:
# 컬럼명을 바꿔주는 함수
def column_rename(df):
    df.columns =  ['회사명', '거래소코드', '회계년도', '액면가(원)', '종가(원)']

# 기초 날짜만 남기는 함수
def first_setting(df):
    df = df[df['회계년도'] <= df['회계년도'][0]]
    df.reset_index(drop=True, inplace=True)
    return df

# 기말 날짜만 남기는 함수
def last_setting(df):
    last_date = df['회계년도'].iloc[-1]
    df = df[df['회계년도'] >= last_date]
    df.reset_index(drop=True, inplace=True)
    return df    

In [286]:
# 기초 데이터만 남기기

df1 = first_setting(df1)
df2 = first_setting(df2)
df5 = first_setting(df5)
df6 = first_setting(df6)
df9 = first_setting(df9)
df10 = first_setting(df10)
df13 = first_setting(df13)
df14 = first_setting(df14)
df17 = first_setting(df17)
df18 = first_setting(df18)
df21 = first_setting(df21)
df22 = first_setting(df22)
df25 = first_setting(df25)
df26 = first_setting(df26)
df29 = first_setting(df29)
df30 = first_setting(df30)
df33 = first_setting(df33)
df34 = first_setting(df34)
df37 = first_setting(df37)
df38 = first_setting(df38)

In [287]:
# 기말 데이터만 남기기

df3 = last_setting(df3)
df4 = last_setting(df4)
df7 = last_setting(df7)
df8 = last_setting(df8)
df11 = last_setting(df11)
df12 = last_setting(df12)
df15 = last_setting(df15)
df16 = last_setting(df16)
df19 = last_setting(df19)
df20 = last_setting(df20)
df23 = last_setting(df23)
df24 = last_setting(df24)
df27 = last_setting(df27)
df28 = last_setting(df28)
df31 = last_setting(df31)
df32 = last_setting(df32)
df35 = last_setting(df35)
df36 = last_setting(df36)
df39 = last_setting(df39)
df40 = last_setting(df40)

In [288]:
# 액면가를 float으로 타입 변환

df_list = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, 
df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35, df36, df37, df38, df39, df40]

for df in df_list:
    df['액면가(원)'] = df['액면가(원)'].astype(float)

In [289]:
df_액면가통합 = pd.concat(df_list)

In [291]:
# 인덱스 정리
df_액면가통합.reset_index(drop=True, inplace=True)

# 회계년도 정리
df_액면가통합['회계년도'] = pd.to_datetime(df_액면가통합['회계년도'])
df_액면가통합['회계년도'] = df_액면가통합['회계년도'].dt.strftime('%Y')

# 가중치 제작
df_액면가통합['가중치'] = 5000 / df_액면가통합['액면가(원)']

# 외국계기업 삭제
df_액면가외국계제외 = df_액면가통합.drop(df_액면가통합['거래소코드'][df_액면가통합['거래소코드'] >= 900000].index)

# 인덱스 정리
df_액면가외국계제외.reset_index(drop=True, inplace=True)

In [293]:
# 가중치의 무한 값 확인
df_액면가외국계제외2 = df_액면가외국계제외.drop(df_액면가외국계제외[df_액면가외국계제외['가중치'] == np.inf].index)

# 인덱스 정리
df_액면가외국계제외2.reset_index(drop=True, inplace=True)

# 수정종가 제작
df_액면가외국계제외2['수정종가'] = df_액면가외국계제외2['종가(원)'] * df_액면가외국계제외2['가중치']

# 그룹화
df_액면가외국계제외3 = (df_액면가외국계제외2.groupby(['거래소코드', '회계년도']).agg(sum) / 2).reset_index()

In [297]:
# 수정종가 제작 완료 ((기초액면가 * 기초종가 + 기말액면가 * 기말종가) / 2)

df_액면가외국계제외3.reset_index(drop=True, inplace=True)
df_액면가외국계제외3

,거래소코드,회계년도,액면가(원),종가(원),가중치,수정종가
0,20,2010,1000.0,6405.0,5.0,32025.0
1,20,2011,1000.0,4680.0,5.0,23400.0
2,20,2012,1000.0,5062.5,5.0,25312.5
3,20,2013,1000.0,5240.0,5.0,26200.0
4,20,2014,1000.0,5005.0,5.0,25025.0
...,...,...,...,...,...,...
18853,337450,2019,50.0,1010.0,25.0,50500.0
18854,339950,2019,50.0,1000.0,25.0,50000.0
18855,340120,2019,50.0,1005.0,25.0,50250.0
18856,340360,2019,50.0,1002.5,25.0,50125.0


In [299]:
# 거래량, 수정종가 결합

df_시장정보 = pd.merge(df_액면가외국계제외3, df_거래량, how='left')

In [305]:
# 회계년도 남기기
df_주식총수['회계년도'] = df_주식총수['회계년도'].str[:4]

# 타입 변경
df_주식총수['회계년도'] = df_주식총수['회계년도'].astype('int64')
df_시장정보['회계년도'] = df_시장정보['회계년도'].astype('int64')

# 필요한 컬럼 남기기
df_주식총수 = df_주식총수[['회계년도', '거래소코드', '발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)']]

In [306]:
# 시장정보 및 주식총수 결합

df_시장정보최종 = pd.merge(df_시장정보, df_주식총수, how='left')

In [307]:
# 시장정보 파생변수 생성

df_시장정보최종['시가총액'] = df_시장정보최종['수정종가'] * df_시장정보최종['발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)']
df_시장정보최종['거래량회전율'] = df_시장정보최종['거래량'] / df_시장정보최종['발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)']
df_시장정보최종['로그시가총액'] = np.log10(df_시장정보최종['시가총액'])
df_시장정보최종['수정거래량'] = df_시장정보최종['거래량'] / df_시장정보최종['수정종가']

In [308]:
# 증가율 파생변수 생성

df_시장정보최종['거래량증가율'] = df_시장정보최종.groupby('거래소코드')['거래량'].pct_change()
df_시장정보최종['시가총액증가율'] = df_시장정보최종.groupby('거래소코드')['시가총액'].pct_change()

In [310]:
df_시장정보최종2 = df_시장정보최종[['거래소코드', '회계년도', '수정종가', '회사명', '시가총액', '거래량회전율', '로그시가총액', '수정거래량', '거래량증가율', '시가총액증가율']]

In [311]:
df_시장정보최종2.to_csv('../datasets/시장정보최종.csv', index=0)

### 8-2. 금융비용부담률, 고정비율, R&D비율, 채무부담비율 제작

In [483]:
# 데이터 로드

df_채무연결 = pd.read_csv('../datasets/채무연결.csv', encoding='cp949')
df_채무개별 = pd.read_csv('../datasets/채무개별.csv', encoding='cp949')


In [485]:
df_채무연결.columns = ['회사명', '거래소코드', '회계년도', '재고자산', '매입채무', '영업활동으로 인한 현금흐름', '매출액(수익)', '이자비용', '유형자산', '자본', '당기순이익(손실)', '연구개발비']
df_채무개별.columns = ['회사명', '거래소코드', '회계년도', '재고자산', '매입채무', '영업활동으로 인한 현금흐름', '매출액(수익)', '이자비용', '유형자산', '자본', '당기순이익(손실)', '연구개발비']

In [486]:
df_채무통합 = df_채무연결.combine_first(df_채무개별)

In [489]:
df_채무통합

,회사명,거래소코드,회계년도,재고자산,매입채무,영업활동으로 인한 현금흐름,매출액(수익),이자비용,유형자산,자본,당기순이익(손실),연구개발비
0,(주)CMG제약,58820,2011/12,2743103.0,1481481.0,-2560438.0,17751704.0,181404.0,9707759.0,18034249.0,-1508282.0,23703.0
1,(주)CMG제약,58820,2012/12,2955172.0,2670886.0,-1572581.0,16255389.0,542880.0,9307482.0,31590685.0,-6660149.0,120052.0
2,(주)CMG제약,58820,2013/12,4618718.0,3911851.0,-3939013.0,20402140.0,1134104.0,10805092.0,48997281.0,296181.0,134454.0
3,(주)CMG제약,58820,2014/12,4913103.0,4719504.0,724893.0,22752584.0,122840.0,18833775.0,49002644.0,158039.0,116533.0
4,(주)CMG제약,58820,2015/12,5455617.0,4844096.0,2410205.0,27041894.0,79830.0,17822468.0,46806888.0,-3650091.0,831910.0
...,...,...,...,...,...,...,...,...,...,...,...,...
18562,흥아해운(주),3280,2015/12,11488626.0,53092211.0,33913401.0,845115359.0,20754125.0,483319774.0,173076634.0,10517216.0,0.0
18563,흥아해운(주),3280,2016/12,17967554.0,49227200.0,-1389623.0,831746081.0,27640109.0,671042170.0,192487245.0,-17173253.0,0.0
18564,흥아해운(주),3280,2017/12,19976759.0,59451213.0,-6892102.0,836427496.0,30888216.0,560312598.0,116884496.0,-73500411.0,0.0
18565,흥아해운(주),3280,2018/12,18807927.0,67579783.0,-33794586.0,753865569.0,32310603.0,492709259.0,73148175.0,-86540156.0,0.0


In [490]:
# 파생변수 제작
df_채무통합['회계년도'] = df_채무통합['회계년도'].str[:4]

df_채무통합['금융비용부담률'] = df_채무통합['이자비용']/df_채무통합['매출액(수익)']*100
df_채무통합.drop(columns=['이자비용'], inplace=True)

df_채무통합['고정비율'] =  df_채무통합['유형자산']/df_채무통합['자본']*100
df_채무통합.drop(columns=['유형자산', '자본', '당기순이익(손실)'], inplace=True)

df_채무통합['R&D비율'] = df_채무통합['연구개발비'] / df_채무통합['매출액(수익)']*100
df_채무통합.drop(columns=['연구개발비', '매출액(수익)'], inplace=True)

df_채무통합['채무부담비율'] = df_채무통합['매입채무']/df_채무통합['재고자산']*100
df_채무통합.drop(columns=['재고자산', '매입채무'], inplace=True)


In [491]:
# 데이터 타입 변경
df_채무통합['회계년도'] = df_채무통합['회계년도'].astype('int64')

# 중복 데이터 제거
df_채무통합.drop_duplicates(keep='first', inplace=True)

# 인덱스 정리
df_채무통합.reset_index(inplace=True,drop=True)

In [492]:
# 데이터 저장
df_채무통합.to_csv('../datasets/비율파생변수.csv', index=0)

### 8-3. DOL, DFL

In [327]:
# 데이터 로드

dol_dfl = pd.read_csv('../datasets/dol_dfl.csv', encoding='cp949')
dol_dfl_2 = pd.read_csv('../datasets/dol_dfl_개별.csv', encoding='cp949')

In [329]:
# 결산월 12월만 남기기
dol_dfl_1 = dol_dfl.drop(dol_dfl[(dol_dfl['회계년도'] != '2010/12') & (dol_dfl['회계년도'] != '2011/12') & (dol_dfl['회계년도'] != '2012/12') & (dol_dfl['회계년도'] != '2013/12') & 
                      (dol_dfl['회계년도'] != '2014/12') & (dol_dfl['회계년도'] != '2015/12') & (dol_dfl['회계년도'] != '2016/12') & 
                      (dol_dfl['회계년도'] != '2017/12') & (dol_dfl['회계년도'] != '2018/12') & (dol_dfl['회계년도'] != '2019/12')].index)
                      
# 결산월 12월만 남기기(개별)
dol_dfl_3 = dol_dfl_2.drop(dol_dfl_2[(dol_dfl_2['회계년도'] != '2010/12') & (dol_dfl_2['회계년도'] != '2011/12') & (dol_dfl_2['회계년도'] != '2012/12') & (dol_dfl_2['회계년도'] != '2013/12') & 
                      (dol_dfl_2['회계년도'] != '2014/12') & (dol_dfl_2['회계년도'] != '2015/12') & (dol_dfl_2['회계년도'] != '2016/12') & 
                      (dol_dfl_2['회계년도'] != '2017/12') & (dol_dfl_2['회계년도'] != '2018/12') & (dol_dfl_2['회계년도'] != '2019/12')].index)

In [330]:
# 회계년도 월 삭제 & 숫자형으로 변환
dol_dfl_1['회계년도'] = dol_dfl_1['회계년도'].apply(lambda x: x.replace('/12', ''))
dol_dfl_1['회계년도'] = dol_dfl_1['회계년도'].astype('int64')

# 회계년도 월 삭제 & 숫자형으로 변환(개별)
dol_dfl_3['회계년도'] = dol_dfl_3['회계년도'].apply(lambda x: x.replace('/12', ''))
dol_dfl_3['회계년도'] = dol_dfl_3['회계년도'].astype('int64')

In [331]:
# 중복값 제거
dol_dfl_1.drop_duplicates(keep='first', inplace=True)

# 중복값 제거(개별)
dol_dfl_3.drop_duplicates(keep='first', inplace=True)

In [332]:
# 컬럼명 변경
dol_dfl_1.columns = ['회사명', '거래소코드', '회계년도', '매출액', '계속영업이익', '당기순이익']

# 컬럼명 변경(개별)
dol_dfl_3.columns = ['회사명', '거래소코드', '회계년도', '매출액', '계속영업이익', '당기순이익']


In [333]:
# 연결, 개별 통합

dol_dfl_통합 = dol_dfl_1.combine_first(dol_dfl_3)

In [334]:
# 파생변수 제작
dol_dfl_통합['영업이익변화율'] = dol_dfl_통합.groupby(['거래소코드'])['계속영업이익'].pct_change()
dol_dfl_통합['매출액변화율'] = dol_dfl_통합.groupby(['거래소코드'])['매출액'].pct_change()

dol_dfl_통합['DOL'] = dol_dfl_통합['영업이익변화율'] / dol_dfl_통합['매출액변화율']
dol_dfl_통합['DFL'] = dol_dfl_통합['계속영업이익'] / dol_dfl_통합['당기순이익']

In [356]:
# 2010년 데이터 삭제
dol_dfl_통합 = dol_dfl_통합[dol_dfl_통합['회계년도'] > 2010]

# 인덱스 정리
dol_dfl_통합.reset_index(drop=True, inplace=True)

In [357]:
# 데이터 저장
dol_dfl_통합.to_csv('../datasets/dol_dfl.csv', index=False)

^^^^^^^
### DOL, DFL 다시 제작함

### 9. 원본데이터에 통합(파생변수, 기타변수)

In [667]:
# 데이터 로드
df_원본 = pd.read_csv('../datasets/3차전처리.csv')
df_change_rate = pd.read_csv('../datasets/증가율_수정.csv')
df_altman = pd.read_csv('../datasets/altman.csv')
df_DOLDFL = pd.read_csv('../datasets/doldfl_final.csv')
df_EV_EBITDA = pd.read_csv('../datasets/EV_EBITDA.csv')
df_비율파생 = pd.read_csv('../datasets/비율파생변수.csv')
df_시장정보데이터 = pd.read_csv('../datasets/시장정보최종.csv')

In [668]:
df_시장정보데이터.drop(columns='회사명', inplace=True)

In [669]:
# 원본데이터에 결합
merge_raw = pd.merge(df_원본, df_change_rate, how='left')
merge_raw2 = pd.merge(merge_raw, df_altman, how='left')
merge_raw3 = pd.merge(merge_raw2, df_DOLDFL, how='left')
merge_raw4 = pd.merge(merge_raw3, df_EV_EBITDA, how='left')
merge_raw5 = pd.merge(merge_raw4, df_비율파생, how='left')
merge_raw6 = pd.merge(merge_raw5, df_시장정보데이터, how='left')

In [670]:
merge_raw6

,회사명,거래소코드,회계년도,자기자본구성비율,설비투자효율,총자본투자효율,이자보상배율(이자비용),유동비율,당좌비율,부채비율,...,고정비율,R&D비율,채무부담비율,수정종가,시가총액,거래량회전율,로그시가총액,수정거래량,거래량증가율,시가총액증가율
0,(주)CMG제약,58820,2011,66.05,89.82,19.56,-9.650000e+00,200.97,165.03,42.33,...,53.829572,0.133525,54.007487,7795.0,3.405885e+11,9.350932,11.532230,52414.645542,2.580317,-0.298137
1,(주)CMG제약,58820,2012,55.32,1749.87,1.66,-1.518000e+01,693.49,649.98,21.50,...,29.462742,0.738537,90.380052,10425.0,6.883163e+11,5.391525,11.837788,34146.601343,-0.128726,1.020962
2,(주)CMG제약,58820,2013,87.63,71.28,34.29,1.920000e+00,694.71,610.74,11.23,...,22.052432,0.659019,84.695602,11385.0,9.862469e+11,1.578308,11.993986,12009.124638,-0.615921,0.432840
3,(주)CMG제약,58820,2014,84.39,79.30,31.04,1.000000e+09,324.58,268.23,17.79,...,38.434202,0.512175,96.059537,9155.0,7.930690e+11,1.535762,11.899311,14531.756854,-0.026957,-0.195872
4,(주)CMG제약,58820,2015,82.80,144.10,15.96,1.000000e+09,296.85,235.21,18.91,...,38.076592,3.076375,88.790984,15200.0,1.342118e+12,8.023355,12.127791,46607.904671,4.325087,0.692309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13936,휴젤(주),145020,2018,85.46,19.27,70.78,1.000000e+09,2971.61,2876.73,2.94,...,7.725415,4.249285,27.716107,4682500.0,2.040729e+13,1.681760,13.309785,1.565286,-0.284462,0.082709
13937,휴젤(주),145020,2019,83.19,29.03,55.32,9.255000e+01,1705.99,1629.03,5.05,...,9.372283,4.998807,19.696592,3842500.0,1.659870e+13,0.818868,13.220074,0.920577,-0.517383,-0.186629
13938,휴젤(주),145020,2019,83.19,29.03,55.32,9.255000e+01,1705.99,1629.03,5.05,...,9.372283,4.998807,19.696592,3842500.0,1.659870e+13,0.818868,13.220074,0.920577,-0.517383,-0.186629
13939,흥아해운(주),3280,2018,9.10,201.90,4.52,-1.830000e+00,34.67,28.66,427.21,...,673.576968,0.000000,359.315426,5785.0,1.016097e+12,3.706850,12.006935,112546.895592,-0.556525,-0.237193


In [671]:
merge_raw6.columns

Index(['회사명', '거래소코드', '회계년도', '자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)',
       '유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '부실', '매출액증가율',
       '총자본증가율', '유동자산증가율', '유형자산증가율', '영업이익증가율', '순이익증가율', '종가', 'RETA',
       'EBTA', 'OM', '종업원수증가율', '매출액', '계속영업이익', '당기순이익', '영업이익변화율', '매출액변화율',
       '당기순이익변화율', 'DOL', 'DFL', 'EV/EBITDA', '영업활동으로 인한 현금흐름', '금융비용부담률',
       '고정비율', 'R&D비율', '채무부담비율', '수정종가', '시가총액', '거래량회전율', '로그시가총액', '수정거래량',
       '거래량증가율', '시가총액증가율'],
      dtype='object')

In [672]:
merge_raw6.drop(columns=['매출액', '계속영업이익', '당기순이익', '수정종가', '종가'], inplace=True)

In [673]:
# 컬럼 순서 정리

merge_raw6 = merge_raw6[['회사명', '거래소코드', '회계년도', '자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)',
'유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
'자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '매출액증가율',
'총자본증가율', '유동자산증가율', '유형자산증가율', '영업이익증가율', '순이익증가율', 'RETA',
'EBTA', 'OM', '종업원수증가율', '영업이익변화율', '매출액변화율',
'당기순이익변화율', 'DOL', 'DFL', 'EV/EBITDA', '영업활동으로 인한 현금흐름', '금융비용부담률',
'고정비율', 'R&D비율', '채무부담비율', '거래량회전율', '로그시가총액', '수정거래량', 
'거래량증가율', '시가총액증가율', '시가총액', '부실']]

In [674]:
df_Raw = merge_raw6.copy()

In [676]:
df_Raw.to_csv('../datasets/4차_전처리.csv', index=0)

### 10. 시가총액 100위 기업 및 결측치 제거

In [687]:
# 회계년도별 데이터 제작
df2011 = df_Raw[df_Raw['회계년도'] == 2011]
df2012 = df_Raw[df_Raw['회계년도'] == 2012]
df2013 = df_Raw[df_Raw['회계년도'] == 2013]
df2014 = df_Raw[df_Raw['회계년도'] == 2014]
df2015 = df_Raw[df_Raw['회계년도'] == 2015]
df2016 = df_Raw[df_Raw['회계년도'] == 2016]
df2017 = df_Raw[df_Raw['회계년도'] == 2017]
df2018 = df_Raw[df_Raw['회계년도'] == 2018]
df2019 = df_Raw[df_Raw['회계년도'] == 2019]

In [688]:
# sort_values를 통해 내림차순 정렬
df2011 = df2011.sort_values(by='시가총액', ascending=False)
df2012 = df2012.sort_values(by='시가총액', ascending=False)
df2013 = df2013.sort_values(by='시가총액', ascending=False)
df2014 = df2014.sort_values(by='시가총액', ascending=False)
df2015 = df2015.sort_values(by='시가총액', ascending=False)
df2016 = df2016.sort_values(by='시가총액', ascending=False)
df2017 = df2017.sort_values(by='시가총액', ascending=False)
df2018 = df2018.sort_values(by='시가총액', ascending=False)
df2019 = df2019.sort_values(by='시가총액', ascending=False)

In [689]:
# 인덱스 정리
df2011.reset_index(inplace=True)
df2012.reset_index(inplace=True)
df2013.reset_index(inplace=True)
df2014.reset_index(inplace=True)
df2015.reset_index(inplace=True)
df2016.reset_index(inplace=True)
df2017.reset_index(inplace=True)
df2018.reset_index(inplace=True)
df2019.reset_index(inplace=True)

In [690]:
# 제거할 리스트 생성
ls_시총100위 = []

for i in range(100):
    ls_시총100위.append(df2011['index'][i])
    ls_시총100위.append(df2012['index'][i])
    ls_시총100위.append(df2013['index'][i])
    ls_시총100위.append(df2014['index'][i])
    ls_시총100위.append(df2015['index'][i])       
    ls_시총100위.append(df2016['index'][i])
    ls_시총100위.append(df2017['index'][i])
    ls_시총100위.append(df2018['index'][i])
    ls_시총100위.append(df2019['index'][i])

In [692]:
df_Raw = df_Raw.drop(ls_시총100위)

In [693]:
# 결측치 제거 전 균형상태 (27.4 : 1)
df_Raw['부실'].value_counts()

0    12582
1      459
Name: 부실, dtype: int64

In [694]:
# 최종 완성 데이터 결측치 제거
df_Raw_drop = df_Raw.dropna()

In [695]:
# 결측치 제거 후 균형상태 (22.8 : 1)
df_Raw_drop['부실'].value_counts()

0    10091
1      441
Name: 부실, dtype: int64

In [696]:
df_Raw_drop.reset_index(drop=True, inplace=True)

In [697]:
# 데이터 저장
df_Raw_drop.to_csv('../datasets/Tabnet_Raw.csv', index=0)

In [698]:
# 데이터 개수(기간 2011~2019년 : 9개년)

print('정상기업의 갯수 :', df_Raw_drop[df_Raw_drop['부실'] == 0]['회사명'].nunique())
print('정상기업의 데이터수 :', len(df_Raw_drop[df_Raw_drop['부실'] == 0]))
print('부실기업의 갯수 :', df_Raw_drop[df_Raw_drop['부실'] == 1]['회사명'].nunique())
print('부실기업의 데이터수 :', len(df_Raw_drop[df_Raw_drop['부실'] == 1]))

정상기업의 갯수 : 1395
정상기업의 데이터수 : 10091
부실기업의 갯수 : 258
부실기업의 데이터수 : 441
